# New Red Bull API

For the Safari Rally Kenya, April 2025, a new API appears to have been introduced.

Now might be a good time to move everything into a database:

- all API calls are used to populate a database;
- all queries are made against the database;
- logic determines whether to upsert items to the database via new API calls.



In [ ]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

from urllib.parse import urljoin
import datetime
from sqlite_utils import Database
from jupyterlite_simple_cors_proxy.cacheproxy import CorsProxy, create_cached_proxy
import os
import sqlite3
from wrc_rallydj.db_table_schemas import SETUP_V2_Q
from pandas import read_sql, DataFrame, json_normalize, merge, pivot, concat, to_datetime


class DatabaseManager:
    def __init__(self, dbname, newdb=False, dbReadOnly=False):
        self.dbname = dbname
        self.conn = self.setup_db(newdb=newdb)
        self.dbReadOnly = dbReadOnly

    def setup_db(self, newdb=False):
        logger.info("Initialising the database...")
        if os.path.isfile(self.dbname) and newdb:
            os.remove(self.dbname)

        if not os.path.isfile(self.dbname):
            newdb = True

        conn = sqlite3.connect(self.dbname, timeout=10)

        if newdb:
            self.initialize_db(conn)

        return conn

    def initialize_db(self, conn):
        logger.info("Creating new db tables...")
        c = conn.cursor()
        c.executescript(SETUP_V2_Q)

    def read_sql(self, query):
        return read_sql(query, self.conn)

    def dbfy(self, df, table, if_exists="upsert", pk=None, index=False, clear=False):
        if self.dbReadOnly:
            return

        if if_exists == "upsert" and not pk:
            return

        if if_exists == "replace":
            clear = True
            if_exists = "append"
        if clear:
            self.cleardbtable(table)

        cols = read_sql(f"PRAGMA table_info({table})", self.conn)["name"].tolist()
        for c in df.columns:
            if c not in cols:
                df.drop(columns=[c], inplace=True)

        logger.info(f"Updating {table}...")

        if if_exists == "upsert":
            DB = Database(self.conn)
            DB[table].upsert_all(df.to_dict(orient="records"), pk=pk)
        else:
            df.to_sql(table, self.conn, if_exists=if_exists, index=index)

    def cleardbtable(self, table):
        c = self.conn.cursor()
        c.execute(f'DELETE FROM "{table}"')


class APIClient:
    RED_BULL_LIVETIMING_API_BASE = (
        "https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/"
    )
    
    def __init__(self, db_manager=None, use_cache=False, **cache_kwargs):
        self.db_manager = db_manager
        self.proxy = create_cached_proxy(**cache_kwargs) if use_cache else CorsProxy()

    def dbfy(self, *args, **kwargs):
        self.db_manager.dbfy(*args, **kwargs)

    def _WRC_RedBull_json(self, path, base=None, retUrl=False):
        """Return JSON from API."""
        base = self.RED_BULL_LIVETIMING_API_BASE if base is None else base
        url = urljoin(base, path)
        print(url)
        if retUrl:
            return url
        # print(f"Fetching: {url}")
        try:
            r = self.proxy.cors_proxy_get(url)
        except:
            print("Error trying to load data.")
            return {}
        # r = requests.get(url)
        rj = r.json()
        if "status" in rj and rj["status"] == "Not Found":
            return {}
        return r.json()

    def _getSeasons(self, updateDB=False):
        """The seasons feed is regularly updated throughout the season."""
        stub = f"seasons.json"
        json_data = self._WRC_RedBull_json(stub)
        seasons_df = DataFrame(json_data)
        if seasons_df.empty:
            return DataFrame()

        if updateDB:
            self.dbfy(seasons_df, "seasons", pk="seasonId")

        return seasons_df

    def _getSeasonDetail(self, seasonId, updateDB=False):
        stub = f"season-detail.json?seasonId={seasonId}"
        json_data = self._WRC_RedBull_json(stub)
        if "championships" not in json_data:
            return DataFrame(), DataFrame(), DataFrame()

        championships_df = DataFrame(json_data["championships"])
        seasonRounds_df = DataFrame(json_data["seasonRounds"])
        _seasonRounds_df = json_normalize(seasonRounds_df["event"])
        _cols = [
            "eventId",
            "countryId",
            "name",
            "slug",
            "location",
            "startDate",
            "finishDate",
            "timeZoneId",
            "timeZoneName",
            "timeZoneOffset",
            "surfaces",
            "organiserUrl",
            "categories",
            "mode",
            "trackingEventId",
            "clerkOfTheCourse",
            "stewards",
            "templateFilename",
            "country.countryId",
            "country.name",
            "country.iso2",
            "country.iso3",
            "seasonId",
            "order",
        ]
        seasonRounds_df = merge(
            _seasonRounds_df, seasonRounds_df[["eventId", "seasonId", "order"]]
        )[_cols]
        # TO DO - improve cleaning
        seasonRounds_df["name"] = seasonRounds_df["name"].str.strip()

        eligibilities_df = json_data["eligibilities"]

        if updateDB:
            self.dbfy(championships_df, "championship_lookup", pk="championshipId")
            self.dbfy(seasonRounds_df, "season_rounds", pk="eventId")

        return championships_df, seasonRounds_df, eligibilities_df

    def _getChampionshipOverallResults(self, championshipId, seasonId, updateDB=False):
        if not championshipId or not seasonId:
            return DataFrame(), DataFrame()

        stub = f"championship-overall-results.json?championshipId={championshipId}&seasonId={seasonId}"
        json_data = self._WRC_RedBull_json(stub)
        if "entryResults" not in json_data:
            return DataFrame(), DataFrame()

        _championshipEntryResultsOverall = []
        _championshipEntryResultsByRound = []

        for championshipEntry in json_data["entryResults"]:
            _championshipEntry = {}
            for k in ["championshipEntryId", "overallPosition", "overallPoints"]:
                _championshipEntry[k] = championshipEntry[k]
            _championshipEntryResultsOverall.append(_championshipEntry)
            _championshipEntryResultsByRound.extend(championshipEntry["roundResults"])

        championshipEntryResultsOverall_df = DataFrame(_championshipEntryResultsOverall)
        championshipEntryResultsByRound_df = DataFrame(_championshipEntryResultsByRound)

        if updateDB:
            self.dbfy(
                championshipEntryResultsOverall_df,
                "championship_overall",
                if_exists="replace",
            )
            self.dbfy(
                championshipEntryResultsByRound_df,
                "championship_results",
                pk=("championshipEntryId", "eventId"),
            )

        return championshipEntryResultsOverall_df, championshipEntryResultsByRound_df

    def _getChampionshipDetail(
        self, year, championship, championshipId=None, seasonId=None, updateDB=False
    ):
        # If championshipId is None, try to find a championship Id
        if not championshipId:

            # Use WRC drivers as the default
            _championship = self._getChampionshipName()

            seasonId, championships_df, _, _ = self._getSeasonDetail()
            championshipId = championships_df[
                championships_df["name"] == _championship
            ].iloc[0]["championshipId"]

        if seasonId is None:
            seasonId = self._getSeasons(championship, year).iloc[0]["seasonId"]

        stub = f"championship-detail.json?championshipId={championshipId}&seasonId={seasonId}"
        json_data = self._WRC_RedBull_json(stub)
        if "championshipRounds" not in json_data:
            return DataFrame(), DataFrame(), DataFrame()

        rounds = [r["event"] for r in json_data["championshipRounds"]]
        championshipRounds_df = DataFrame(rounds)
        championshipRounds_df["year"] = to_datetime(
            championshipRounds_df["startDate"]
        ).dt.year
        championshipCountries_df = json_normalize(
            championshipRounds_df["country"]
        ).drop_duplicates()
        championshipRounds_df.drop(columns=["country"], inplace=True)

        renamers = {
            k.replace("Description", ""): json_data[k]
            for k in json_data.keys()
            if k.startswith("field") and k.endswith("Description")
        }

        _e = json_data["championshipEntries"]
        championshipEntries_df = DataFrame(_e)
        renamers["tyreManufacturer"] = "tyreManufacturerId"
        championshipEntries_df.rename(columns=renamers, inplace=True)

        if updateDB:
            self.dbfy(
                championshipRounds_df,
                "championship_rounds_detail",
                pk="eventId",
            )
            self.dbfy(
                championshipCountries_df,
                "championship_countries",
                pk="countryId",
            )
            self.dbfy(
                championshipEntries_df,
                "championship_entries",
                pk="championshipEntryId",
            )

        return championshipRounds_df, championshipEntries_df, championshipCountries_df

    def _getEvent(self, eventId, updateDB=False):
        stub = f"events/{eventId}.json"
        json_data = self._WRC_RedBull_json(stub)
        if "rallies" not in json_data:
            return DataFrame(), DataFrame(), DataFrame()

        eventRallies_df = DataFrame(json_data["rallies"])
        eventRallies_df.drop(columns=["eventClasses"], inplace=True)
        eventClasses_df = DataFrame(json_data["eventClasses"])
        _data = {
            k: json_data[k] for k in json_data if k not in ["rallies", "eventClasses"]
        }
        eventData_df = json_normalize(_data)

        if updateDB:
            self.dbfy(eventClasses_df, "event_classes", pk=("eventId", "eventClassId"))
            self.dbfy(eventRallies_df, "event_rallies", pk="itineraryId")
            self.dbfy(eventData_df, "event_date", pk="eventId")

        return eventData_df, eventRallies_df, eventClasses_df

    def _getEntries(self, eventId, rallyId, updateDB=False):
        stub = f"events/{eventId}/rallies/{rallyId}/entries.json"
        json_data = self._WRC_RedBull_json(stub)
        entries_df = DataFrame(json_data)
        if entries_df.empty:
            return (
                DataFrame(),
                DataFrame(),
                DataFrame(),
                DataFrame(),
                DataFrame(),
                DataFrame(),
                DataFrame(),
            )

        entries_df["rallyId"] = rallyId
        drivers_df = json_normalize(entries_df["driver"])
        codrivers_df = json_normalize(entries_df["codriver"])
        manufacturers_df = json_normalize(entries_df["manufacturer"]).drop_duplicates()
        entrants_df = json_normalize(entries_df["entrant"]).drop_duplicates()
        entryGroups_df = json_normalize(entries_df["group"]).drop_duplicates()
        eventClasses_df = json_normalize(
            entries_df.explode("eventClasses")["eventClasses"]
        ).drop_duplicates()

        entries_df.drop(
            columns=[
                "driver",
                "codriver",
                "manufacturer",
                "entrant",
                "group",
                "eventClasses",
                "tags",
            ],
            inplace=True,
        )

        if updateDB:
            self.dbfy(entries_df, "entries", pk="entryId")
            self.dbfy(drivers_df, "entries_drivers", pk="personId")
            self.dbfy(codrivers_df, "entries_codrivers", pk="personId")
            self.dbfy(entryGroups_df, "groups", pk="groupId")
            self.dbfy(manufacturers_df, "manufacturers", pk="manufacturerId")
            self.dbfy(entrants_df, "entrants", pk="entrantId")

        return (
            entries_df,
            drivers_df,
            codrivers_df,
            manufacturers_df,
            entrants_df,
            entryGroups_df,
            eventClasses_df,
        )

    def _getStartLists(self, eventId, startListId=None, updateDB=False):
        if not startListId:
            return DataFrame()

        stub = f"events/{eventId}/startLists/{startListId}.json"
        json_data = self._WRC_RedBull_json(stub)

        startlist_df = json_normalize(json_data["startListItems"])
        startlist_df["eventId"] = json_data["eventId"]
        startlist_df["name"] = json_data["name"]

        if updateDB:
            self.dbfy(startlist_df, "startlists", pk="startListItemId")

        return startlist_df

    def _getEventGroups(self, eventId, updateDB=False):
        stub = f"events/{eventId}/groups.json"
        json_data = self._WRC_RedBull_json(stub)

        eventGroups_df = DataFrame(json_data)
        if eventGroups_df.empty:
            return DataFrame()

        if updateDB:
            self.dbfy(eventGroups_df, "groups", pk="groupId")
        return eventGroups_df

    def _getEventItineraries(self, eventId, itineraryId, updateDB=False):
        stub = f"events/{eventId}/itineraries/{itineraryId}.json"
        json_data = self._WRC_RedBull_json(stub)
        if "itineraryLegs" not in json_data:
            return DataFrame(), DataFrame(), DataFrame(), DataFrame()

        itineraryLegs_df = DataFrame(json_data["itineraryLegs"])
        itineraryLegs_df["eventId"] = eventId

        if "itinerarySections" not in itineraryLegs_df:
            return itineraryLegs_df, DataFrame(), DataFrame(), DataFrame()

        itinerarySections_df = itineraryLegs_df.explode("itinerarySections")

        itinerarySections2_df = json_normalize(
            itinerarySections_df["itinerarySections"]
        )

        itineraryControls_df = itinerarySections2_df.explode("controls").reset_index(
            drop=True
        )
        _itineraryControls_df = json_normalize(itineraryControls_df["controls"])
        itineraryControls_df = concat(
            [itineraryControls_df.drop("controls", axis=1), _itineraryControls_df],
            axis=1,
        )
        itineraryStages_df = itinerarySections2_df.explode("stages").reset_index(
            drop=True
        )

        itineraryStages_df.rename(columns={"name": "name_"}, inplace=True)
        _itineraryStages_df = json_normalize(itineraryStages_df["stages"])
        itineraryStages_df = concat(
            [itineraryStages_df.drop("stages", axis=1), _itineraryStages_df], axis=1
        )

        itinerarySections2_df["eventId"] = eventId
        
        itineraryLegs_df.drop(columns=["itinerarySections"], inplace=True)
        itinerarySections2_df.drop(columns=["controls", "stages"], inplace=True)
        itineraryControls_df.drop(columns=["stages"], inplace=True)
        itineraryStages_df.drop(columns=["controls"], inplace=True)

        if updateDB:
            self.dbfy(itineraryLegs_df, "itinerary_legs", pk="itineraryLegId")
            self.dbfy(itineraryStages_df, "itinerary_stages", pk="stageId")
            self.dbfy(
                itinerarySections2_df, "itinerary_sections", pk="itinerarySectionId"
            )
            self.dbfy(itineraryControls_df, "itinerary_controls", pk="controlId")
            for _, row in itineraryLegs_df.iterrows():
                startListId = row["startListId"]
                self._getStartLists(
                    eventId=eventId, startListId=startListId, updateDB=updateDB
                )

        return (
            itineraryLegs_df,
            itinerarySections2_df,
            itineraryControls_df,
            itineraryStages_df,
        )

    def _getControlTimes(self, eventId, controlId, updateDB=False):
        stub = f"events/{eventId}/controls/{controlId}/controlTimes.json"
        json_data = self._WRC_RedBull_json(stub)
        controlTimes_df = DataFrame(json_data)
        if controlTimes_df.empty:
            return DataFrame()

        if updateDB:
            self.dbfy(controlTimes_df, "controltimes", pk="controlTimeId")

        return controlTimes_df

    def _getEventShakeDownTimes(self, eventId, updateDB=False):
        if not eventId:
            return
        stub = f"events/{eventId}/shakedowntimes.json?shakedownNumber=1"
        json_data = self._WRC_RedBull_json(stub)
        shakedownTimes_df = DataFrame(json_data)

        if updateDB:
            self.dbfy(shakedownTimes_df, "shakedown_times", pk="shakedownTimeId")

        return shakedownTimes_df

    def _getStages(self, eventId, updateDB=False):
        stub = f"events/{eventId}/stages.json"
        json_data = self._WRC_RedBull_json(stub)
        stages_df = DataFrame(json_data)

        if stages_df.empty:
            return (DataFrame(), DataFrame(), DataFrame())

        stage_split_points_df = (
            stages_df[["splitPoints"]].explode("splitPoints").reset_index(drop=True)
        )
        stage_controls_df = json_normalize(
            stages_df[["controls"]]
            .explode("controls")
            .reset_index(drop=True)["controls"]
        )
        stage_split_points_df = json_normalize(
            stages_df[["splitPoints"]]
            .explode("splitPoints")
            .reset_index(drop=True)["splitPoints"]
        )
        
        stages_df.drop(columns=["splitPoints", "controls"], inplace=True)

        if updateDB:
            self.dbfy(stages_df, "stage_info", pk="stageId")
            self.dbfy(stage_split_points_df, "split_points", pk="splitPointId")
            self.dbfy(stage_controls_df, "stage_controls", pk="controlId")

        return stages_df, stage_split_points_df, stage_controls_df

    def _getStageTimes(self, eventId, rallyId, stageId=None, updateDB=False):
        stub = f"events/{eventId}/stages/{stageId}/stagetimes.json?rallyId={rallyId}"
        json_data = self._WRC_RedBull_json(stub)
        stagetimes_df = DataFrame(json_data)
        if stagetimes_df.empty:
            return DataFrame()

        stagetimes_df["eventId"] = eventId
        stagetimes_df["rallyId"] = rallyId

        if updateDB:
            self.dbfy(stagetimes_df, "stage_times", pk="stageTimeId")

        return stagetimes_df

    def _getSplitTimes(self, eventId, rallyId, stageId=None, updateDB=False):
        stageId = stageId if stageId else self.stageId
        stub = f"events/{eventId}/stages/{stageId}/splittimes.json?rallyId={rallyId}"
        json_data = self._WRC_RedBull_json(stub)
        splitTimes_df = DataFrame(json_data)
        if splitTimes_df.empty:
            return DataFrame()

        splitTimes_df["stageId"] = stageId
        splitTimes_df["eventId"] = eventId
        splitTimes_df["rallyId"] = rallyId

        if updateDB:
            self.dbfy(splitTimes_df, "split_times", pk="splitPointTimeId")

        return splitTimes_df

    def _getStageOverallResults(
        self,
        eventId,
        rallyId,
        championshipId=None,
        stageId=None,
        by_championship=False,
        updateDB=False,
    ):
        """This is the overall result at the end of the stage. TO DO CHECK"""
        # TO DO: if we select by championship, are these different?
        # If so, do we need to set championship and championship Pos cols, maybe in a new table?
        # The rallyId is optional? Or does it filter somehow?
        stageId = stageId if stageId else self.stageId
        stub = f"events/{eventId}/stages/{stageId}/results.json?rallyId={rallyId}"
        if by_championship and championshipId:
            stub = stub + f"&championshipId={championshipId}"
        json_data = self._WRC_RedBull_json(stub)
        stageResults_df = DataFrame(json_data)
        if stageResults_df.empty:
            return DataFrame()

        stageResults_df["stageId"] = stageId
        stageResults_df["eventId"] = eventId
        stageResults_df["rallyId"] = rallyId

        if updateDB:
            self.dbfy(stageResults_df, "stage_overall", pk=("stageId", "entryId"))

        return stageResults_df

    def _getStageWinners(self, eventId, rallyId, updateDB=False):
        stub = f"events/{eventId}/rallies/{rallyId}/stagewinners.json"
        json_data = self._WRC_RedBull_json(stub)
        stagewinners_df = DataFrame(json_data)
        if stagewinners_df.empty:
            return DataFrame()

        stagewinners_df["eventId"] = eventId
        stagewinners_df["rallyId"] = rallyId
        if updateDB:
            self.dbfy(stagewinners_df, "stagewinners", pk="stageId")

        return stagewinners_df

    def _getRetirements(self, eventId, updateDB=False):
        if not eventId:
            return
        stub = f"events/{eventId}/retirements.json"
        json_data = self._WRC_RedBull_json(stub)
        retirements_df = DataFrame(json_data)
        if retirements_df.empty:
            return DataFrame()

        retirements_df["eventId"] = eventId
        if updateDB:
            self.dbfy(retirements_df, "retirements", pk="retirementId")
        return retirements_df

    def _getPenalties(self, eventId, updateDB=False):
        if not eventId:
            return
        stub = f"events/{eventId}/penalties.json"
        json_data = self._WRC_RedBull_json(stub)
        penalties_df = DataFrame(json_data)
        if penalties_df.empty:
            return DataFrame()

        penalties_df["eventId"] = eventId
        if updateDB:
            self.dbfy(penalties_df, "penalties", pk="penaltyId")
        return penalties_df


# The WRCTimingResultsAPIClientV2() constructs state on a season basis
class WRCTimingResultsAPIClientV2:
    CHAMPIONSHIP_CODES = {"World Rally Championship":"wrc", "European Rally Championship":"erc"}
    SPLIT_PREFIX = "SP"
    SPLIT_FINAL = "FINAL"

    def __init__(
        self,
        year: int = datetime.date.today().year,
        championship: str = "wrc", # wrc | erc
        category: str = "Drivers",
        group: str = "all",
        dbname: str = "wrcRbAPITiming.db",
        dbReadOnly: bool = False,
        newDB: bool = False,
        use_cache: bool = False,
        **cache_kwargs,
    ):
        # Initialize the proxy with caching if requested
        if use_cache:
            self.proxy = create_cached_proxy(**cache_kwargs)
        else:
            self.proxy = CorsProxy()

        self.year = year
        # TO DO - more logic yet surrounding championship
        self.championship = championship  # Informal: WRC, WRC2, WRC3, JWRC
        self.championshipLookup ={}
        self.category = category
        self.seasonId = None
        self.rallyId = None
        self.eventId = None
        self.eventName = None
        self.championshipId = None
        self.championshipName = None
        self.itineraryId = None
        self.stageId = None
        self.stageName = None
        self.stageCode = None

        # DB setup
        self.dbReadOnly = dbReadOnly
        self.db_manager = DatabaseManager(dbname, newdb=newDB, dbReadOnly=dbReadOnly)

        self.api_client = APIClient(
            db_manager=self.db_manager, use_cache=use_cache, **cache_kwargs
        )

        # DB initialise
        if newDB:
            self.seedDB()

    def seedDB(self):
        # Populate the database with seasons info
        # self._getSeasons(updateDB=True)
        # Initialise the seasonId
        _seasons = self._getSeasons(updateDB=True)
        self.seasonId = self._getSeasonsSubQuery(
            _seasons, self.championship, self.year
        ).iloc[0]["seasonId"]
        # Update the season detail
        self._getSeasonDetail(updateDB=True)

    def initialise(self, year=None, championship=None):
        if year:
            self.year = year
        if championship:
            self.championship = championship

        self.getChampionships(updateDB=True)

        # In the initialise phase, take the opportunity to update the season table
        # whilst also setting self.seasonId
        self.setSeason()

        # Set self.championshipId
        self.setChampionship()

    def dbfy(self, *args, **kwargs):
        self.db_manager.dbfy(*args, **kwargs)

    def _WRC_RedBull_json(self, *args, **kwargs):
        return self.api_client._WRC_RedBull_json(*args, **kwargs)

    @staticmethod
    def subtract_from_rows(df, colsList, ignore_first_row=True):
        """
        Subtracts the values of specified columns in the first row from all rows except the first.
        Modifies the DataFrame in place.

        Parameters:
        df (DataFrame): The DataFrame to modify.
        colsList (list): List of column names to subtract.
        """
        df = df.copy()
        df.loc[int(ignore_first_row) :, colsList] -= df.loc[0, colsList].values.astype(
            float
        )  # Perform subtraction directly
        return df

    @staticmethod
    def rebaseTimes(times, rebaseId=None, idCol=None, rebaseCol=None):
        """Rebase times based on the time for a particular vehicle."""
        if not rebaseId or rebaseId == "NONE" or idCol is None or rebaseCol is None:
            return times
        return times[rebaseCol] - times.loc[times[idCol] == rebaseId, rebaseCol].iloc[0]

    @staticmethod
    def rebaseManyTimes(
        times, rebaseId=None, idCol=None, rebaseCols=None, inplace=False
    ):
        """Rebase times in several specified columns relative to a particular vehicle."""
        if not inplace:
            if not rebaseId or rebaseId == "NONE":
                return times
            times = times.copy()

        if rebaseId and rebaseId != "NONE":
            # Ensure rebaseCols is a list
            rebaseCols = [rebaseCols] if isinstance(rebaseCols, str) else rebaseCols

            # Fetch the reference values for the specified 'rebaseId'
            reference_values = times.loc[times[idCol] == rebaseId, rebaseCols].iloc[0]

            # Subtract only the specified columns
            times[rebaseCols] = times[rebaseCols].subtract(reference_values)

            if not inplace:
                return times

    @staticmethod
    def rebaseWithDummyValues(times, replacementVals, rebaseCols=None):
        """
        Add a dummy row, rebase the values, then remove the dummy row.

        :param times: DataFrame containing the data to be modified
        :param replacementVals: List of values to replace for each column in rebaseCols
        :param rebaseCols: List of columns to apply the rebase operation
        :return: Modified DataFrame with rebased values
        """
        if rebaseCols is None:
            return times
        times = times.copy()
        # TO DO:
        # should we have a generic checker that rebase cols are available
        # or subset to the ones that are?
        # If rebaseCols is not a list, make it a list
        rebaseCols = [rebaseCols] if isinstance(rebaseCols, str) else rebaseCols

        # Ensure replacementValsList is the same length as rebaseCols
        if len(replacementVals) != len(rebaseCols):
            raise ValueError(
                "replacementValsList must have the same length as rebaseCols"
            )

        # Create a dummy row with the replacement values
        dummy_row = {col: val for col, val in zip(rebaseCols, replacementVals)}

        # Append the dummy row to the DataFrame
        times = times.append(dummy_row, ignore_index=True)

        # Rebase using the dummy row (rebase the last row in the DataFrame)
        times[rebaseCols] = times[rebaseCols].subtract(
            times.loc[times.index[-1], rebaseCols]
        )

        # Remove the dummy row
        times = times.drop(times.index[-1])

        return times

    def _getSeasonsSubQuery(self, seasons_df, championship=None, year=None):
        if championship is not None:
            if championship.lower() == "wrc":
                championship = "World Rally Championship"
            elif championship.lower() == "erc":
                championship = "European Rally Championship"
            seasons_df = seasons_df[seasons_df["name"] == championship]

        if year is not None:
            seasons_df = seasons_df[seasons_df["year"] == year]
        return seasons_df

    def _getSeasons(self, *args, **kwargs):
        return self.api_client._getSeasons(*args, **kwargs)

    def getSeasons(self, championship=None, year=None, updateDB=False):
        if updateDB:
            self._getSeasons(updateDB=updateDB)
        # TO DO need to filter with championship and year
        q = "SELECT * FROM seasons;"
        seasons_df = self.db_manager.read_sql(q)

        return self._getSeasonsSubQuery(seasons_df, championship, year)

    # TO DO Need to check this e.g. for WRC and ERC
    # TO DO if we set from seasonId,
    def setSeason(self, seasonId=None):
        _seasons = self._getSeasons(updateDB=True)
        if seasonId:
            self.seasonId=seasonId
        else:
            self.seasonId = self._getSeasonsSubQuery(
                _seasons, self.championship, self.year
            ).iloc[0]["seasonId"]

    # This datafeed is partial at the start of the season
    # and needs to be regularly updated
    def _getSeasonDetail(self, *args, **kwargs):
        kwargs["seasonId"] = self.seasonId
        return self.api_client._getSeasonDetail(*args, **kwargs)

    def getSeasonRounds(self, seasonId=None, on_season=True, updateDB=False):
        if updateDB:
            self._getSeasonDetail(updateDB=updateDB)
        if on_season:
            seasonId = seasonId if seasonId else self.seasonId
        seasonId_ = f"""AND seasonId={seasonId}""" if seasonId else ""

        q = f"SELECT * FROM season_rounds WHERE 1=1 {seasonId_};"
        seasonRounds_df = self.db_manager.read_sql(q)
        if seasonRounds_df.empty:
            self._getSeasonDetail(updateDB=True)
            seasonRounds_df = self.db_manager.read_sql(q)

        return seasonRounds_df

    def _getChampionshipName(self):
        championship = self.championship
        categories = ["Drivers", "Co-Drivers"]
        if self.championship.lower() in ["wrc", "wrc2"]:
            categories.append("Teams")
        if self.championship.lower() in ["wrc"]:
            categories.append("Manufacturers")
        category = self.category
        # Category: Drivers, Co-Drivers, Manufacturers, Teams
        if category not in categories:
            return None

        # TO DO - different for ERC
        # TO DO need a muchg better way of handling this that also works with ERC
        # TO DO - are we actually making use of this?
        _championship = "FIA World Rally Championship for Drivers"
        if championship.lower() == "wrc":
            _championship = f"FIA World Rally Championship for {category}"
        elif championship.lower() == "wrc2":
            _championship = f"FIA WRC2 Championship for {category}"
        elif championship.lower() == "wrc3":
            _championship = f"FIA WRC3 Championship for {category}"
        elif championship.lower() == "jwrc":
            _championship = f"FIA Junior WRC Championship for {category}"
        elif championship.lower() == "challenger":
            _championship = f"FIA WRC2 Challenger Championship for {category}"
        elif championship.lower() == "masters":
            _championship = f"FIA WRC Masters Cup for {category}"
        return _championship

    def _getChampionshipOverallResults(self, *args, **kwargs):
        kwargs["championshipId"] = self.championshipId
        kwargs["seasonId"] = self.seasonId
        return self.api_client._getChampionshipOverallResults(*args, **kwargs)

    def getChampionshipOverall(self, updateDB=False):
        if updateDB:
            self._getChampionshipOverallResults(updateDB=updateDB)

        q = "SELECT * FROM championship_overall;"
        championshipEntryResultsOverall_df = self.db_manager.read_sql(q)

        return championshipEntryResultsOverall_df

    def getChampionshipByRound(
        self,
        championshipId=None,
        eventId=None,
        on_event=False,
        on_championship=False,
        updateDB=False,
    ):
        if updateDB:
            self._getChampionshipOverallResults(updateDB=updateDB)
        if championshipId or on_championship:
            if on_championship and self.championshipId:
                championshipId = self.championshipId
            championship_ = f"""cr.championshipId={championshipId}"""
        else:
            championship_ = ""
        if on_event or eventId:
            # TO DO what is best logic if noth are set?
            if on_event and self.eventId:
                eventId = self.eventId
            event_ = f"AND cr.eventId={eventId}"
        else:
            event_ = ""
        q = f"""SELECT * FROM championship_results AS cr WHERE 1=1 {championship_} {event_};"""
        championshipEntryResultsByRound_df = self.db_manager.read_sql(q)

        return championshipEntryResultsByRound_df

    def _getChampionshipDetail(self, *args, **kwargs):
        kwargs["year"] = self.year
        kwargs["championship"] = self.championship
        kwargs["championshipId"] = self.championshipId
        kwargs["seasonId"] = self.seasonId
        return self.api_client._getChampionshipDetail(*args, **kwargs)

    def getChampionShipRounds(self, year=None, updateDB=False):
        if updateDB:
            self._getChampionshipDetail()

        year_ = f"""AND cd.year={year}""" if year else ""
        q = f"""SELECT * FROM championship_rounds_detail AS cd WHERE 1=1 {year_};"""
        championshipRounds_df = self.db_manager.read_sql(q)

        return championshipRounds_df

    def getChampionshipEntries(
        self, championshipId=None, on_championship=False, updateDB=False
    ):
        if updateDB:
            self._getChampionshipDetail()
        if championshipId or on_championship:
            if on_championship and self.championshipId:
                championshipId = self.championshipId
            championship_ = f"""ce.championshipId={championshipId}"""
        else:
            championship_ = ""
        q = f"""SELECT * FROM championship_entries AS ce WHERE 1=1 {championship_};"""
        championshipEntries_df = self.db_manager.read_sql(q)

        return championshipEntries_df

    def getChampionshipCountries(self, updateDB=False):
        if updateDB:
            self._getChampionshipDetail()

        q = "SELECT * FROM championship_countries;"
        championships_df = self.db_manager.read_sql(q)

        return championships_df

    def getChampionships(self, updateDB=False):
        if updateDB:
            self._getSeasonDetail(updateDB=updateDB)

        q = "SELECT * FROM championship_lookup;"
        championshipCountries_df = self.db_manager.read_sql(q)

        return championshipCountries_df

    def _getEventGroups(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        return self.api_client._getEventGroups(*args, **kwargs)

    def _getEventItineraries(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        kwargs["itineraryId"] = self.itineraryId
        return self.api_client._getEventItineraries(*args, **kwargs)

    def getItineraryLegs(self, eventId=None, updateDB=False):
        if updateDB:
            self._getEventItineraries(updateDB=updateDB)

        if eventId is None:
            q = "SELECT * FROM itinerary_legs;"
        else:
            q = f"SELECT * FROM itinerary_legs WHERE eventId={int(eventId)};"

        itineraryLegs_df = self.db_manager.read_sql(q)

        return itineraryLegs_df

    def getItineraryStages(
        self, eventId=None, itineraryLegId=None, itinerarySectionId=None, updateDB=False
    ):
        if updateDB:
            self._getEventItineraries(updateDB=updateDB)

        if not eventId and not itineraryLegId and not itinerarySectionId:
            q = "SELECT * FROM itinerary_stages;"
        elif itinerarySectionId:
            q = f"SELECT * FROM itinerary_stages WHERE itinerarySectionId={int(itinerarySectionId)};"
        elif itineraryLegId:
            q = f"SELECT * FROM itinerary_stages WHERE itineraryLegId={int(itineraryLegId)};"
        elif eventId:
            q = f"SELECT * FROM itinerary_stages WHERE eventId={int(eventId)};"

        itinerarySections_df = self.db_manager.read_sql(q)

        return itinerarySections_df

    def getItinerarySections(self, eventId=None, itineraryLegId=None, updateDB=False):
        if updateDB:
            self._getEventItineraries(updateDB=updateDB)

        if not eventId and not itineraryLegId:
            q = "SELECT * FROM itinerary_sections;"
        elif itineraryLegId:
            q = f"SELECT * FROM itinerary_sections WHERE itineraryLegId={int(itineraryLegId)};"
        elif eventId:
            q = f"SELECT * FROM itinerary_sections WHERE eventId={int(eventId)};"

        itinerarySections_df = self.db_manager.read_sql(q)

        return itinerarySections_df

    def getItineraryControls(
        self, eventId=None, itineraryLegId=None, itinerarySectionId=None, updateDB=False
    ):
        if updateDB:
            self._getEventItineraries(updateDB=updateDB)

        if not eventId and not itineraryLegId and not itinerarySectionId:
            q = "SELECT * FROM itinerary_controls;"
        elif itinerarySectionId:
            q = f"SELECT * FROM itinerary_controls WHERE itinerarySectionId={int(itinerarySectionId)};"
        elif itineraryLegId:
            q = f"SELECT * FROM itinerary_controls WHERE itineraryLegId={int(itineraryLegId)};"
        elif eventId:
            q = f"SELECT * FROM itinerary_controls WHERE eventId={int(eventId)};"

        itineraryControls_df = self.db_manager.read_sql(q)

        return itineraryControls_df

    def _getStartLists(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        return self.api_client._getStartLists(*args, **kwargs)

    def getStartList(self, eventId=None, startListId=None, raw=True, updateDB=False):
        # TO DO  - offer e.g. a day instead of startListId

        if updateDB and startListId:
            self._getStartLists(startListId=startListId)

        eventId = eventId if eventId else self.eventId
        _on_event = f"sl.eventId={eventId}" if eventId else "1=1"
        if raw:
            q = f"SELECT * FROM startlists AS sl WHERE {_on_event}"
            if startListId:
                q = f"{q} AND startListId={startListId}"
            q = f"{q};"
        else:
            _entry_join = f"INNER JOIN entries AS e ON sl.entryId=e.entryId"
            _driver_join = f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
            _codriver_join = (
                f"INNER JOIN entries_codrivers AS cd ON e.codriverId=cd.personId"
            )
            _manufacturer_join = (
                f"INNER JOIN manufacturers AS m ON e.manufacturerId=m.manufacturerId"
            )
            _entrants_join = f"INNER JOIN entrants AS n ON e.entrantId=n.entrantId"
            q = f"SELECT d.fullName AS driverName, cd.fullName AS codriverName, m.name AS manufacturerName, n.name AS entrantName, e.identifier AS carNo, e.vehicleModel, e.priority, e.eligibility, sl.* FROM startlists AS sl {_entry_join} {_driver_join} {_codriver_join} {_manufacturer_join} {_entrants_join} WHERE {_on_event}"
            if startListId:
                q = f"{q} AND startListId={startListId};"
            q = f"{q} ORDER BY sl.[order] ASC;"
            # We can also escape order (reserved word as column name)
            # using eg sl.[order] or sl."order" or  sl.`order`

        startlist_df = self.db_manager.read_sql(q)

        return startlist_df

    def _getEntries(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        kwargs["rallyId"] = self.rallyId

        return self.api_client._getEntries(*args, **kwargs)

    def getEntries(self, on_event=True, updateDB=False):
        if updateDB:
            self._getEntries(updateDB=updateDB)
        _on_event = (
            f"AND eventId={self.eventId} AND rallyId={self.rallyId}"
            if on_event and self.eventId and self.rallyId
            else ""
        )
        q = f"SELECT * FROM entries AS e WHERE 1=1 {_on_event};"
        entries_df = self.db_manager.read_sql(q)

        return entries_df

    def getDrivers(
        self, on_event=True, by_championship=False, priority=None, updateDB=False
    ):
        if updateDB:
            self._getEntries(updateDB=updateDB)
        _on_event = (
            f"AND e.eventId={self.eventId} AND e.rallyId={self.rallyId}"
            if on_event and self.eventId and self.rallyId
            else ""
        )
        _entry_join = f"INNER JOIN entries AS e ON d.personId=e.driverId"
        # TO DO - handle priority
        priority = None if priority == "P0" else priority
        priority_ = f"""AND e.priority="{priority}" """ if priority else ""

        # I'm not sure this is right? Is the championshipId persistent over years?
        # For WRC, use priority (P1) and / or eligibility includes M? Or groupId for Rally1?
        # We want people registered in the championship when they are competing in an event.
        # We can perhaps get IDs by using the championshipId filtered results.json?
        by_championship_ = ""
        if by_championship and self.championshipId:
            by_championship_ = f"AND e.driverId IN (SELECT personId FROM championship_entries WHERE championshipId={self.championshipId})"

        q = f"SELECT d.* FROM entries_drivers AS d {_entry_join} WHERE 1=1 {_on_event} {priority_} {by_championship_};"
        drivers_df = self.db_manager.read_sql(q)

        return drivers_df

    def getCoDrivers(self, on_event=True, updateDB=False):
        if updateDB:
            self._getEntries(updateDB=updateDB)
        _on_event = (
            f"INNER JOIN entries AS e ON cd.personId=e.codriverId WHERE e.eventId={self.eventId} AND e.rallyId={self.rallyId}"
            if on_event and self.eventId and self.rallyId
            else ""
        )

        q = f"SELECT cd.* FROM entries_codrivers AS cd {_on_event};"
        codrivers_df = self.db_manager.read_sql(q)

        return codrivers_df

    def getManufacturers(self, on_event=True, updateDB=False):
        if updateDB:
            self._getEntries(updateDB=updateDB)
        _on_event = (
            f"INNER JOIN entries AS e ON m.manufacturerId=e.manufacturerId WHERE e.eventId={self.eventId} AND e.rallyId={self.rallyId}"
            if on_event and self.eventId and self.rallyId
            else ""
        )

        q = f"SELECT DISTINCT m.* FROM manufacturers AS m {_on_event};"
        manufacturers_df = self.db_manager.read_sql(q)

        return manufacturers_df

    def getEntrants(self, on_event=True, updateDB=False):
        if updateDB:
            self._getEntries(updateDB=updateDB)
        _on_event = (
            f"INNER JOIN entries AS e ON n.entrantId=e.entrantId WHERE e.eventId={self.eventId} AND e.rallyId={self.rallyId}"
            if on_event and self.eventId and self.rallyId
            else ""
        )

        q = f"SELECT n.entrantId, n.name FROM entrants AS n {_on_event};"
        entrants_df = self.db_manager.read_sql(q)

        return entrants_df

    def getGroups(self, on_event=True, updateDB=False):
        if updateDB:
            self._getEntries(updateDB=updateDB)
        _on_event = f"" if on_event else ""

        q = f"SELECT * FROM groups;"
        entryGroups_df = self.db_manager.read_sql(q)

        return entryGroups_df

    def _getEventShakeDownTimes(self, *args, stageId=None, **kwargs):
        kwargs["eventId"] = self.eventId

        return self.api_client._getS_getEventShakeDownTimestages(*args, **kwargs)

    def _getStages(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId

        return self.api_client._getStages(*args, **kwargs)

    def getStageInfo(
        self,
        on_event=True,
        itineraryLegId=None,
        itinerarySectionId=None,
        stage_code=None,
        raw=True,
        updateDB=False,
    ):
        if updateDB:
            self._getStages(updateDB=updateDB)

        on_event_ = f"""si.eventId={self.eventId}""" if on_event else "1=1"
        on_leg_ = (
            f"""AND it_l.itineraryLegId={itineraryLegId}""" if itineraryLegId else ""
        )
        on_section_ = (
            f"""AND it_se.itinerarySectionId={itinerarySectionId}"""
            if itinerarySectionId
            else ""
        )

        if stage_code and isinstance(stage_code, str):
            stage_code = [stage_code]
        else:
            stage_code = []

        if raw:
            q = f"SELECT * FROM stage_info AS si WHERE {on_event_};"
        else:
            _itinerary_stages_join = (
                f"INNER JOIN itinerary_stages AS it_st ON it_st.stageId=si.stageId"
            )
            _itinerary_sections_join = f"INNER JOIN itinerary_sections AS it_se ON it_se.itinerarySectionId=it_st.itinerarySectionId"
            _itinerary_legs_join = f"INNER JOIN itinerary_legs AS it_l ON it_l.itineraryLegId=it_st.itineraryLegId"
            q = f"SELECT it_se.name AS sectionName, it_l.name AS day, si.* FROM stage_info AS si {_itinerary_stages_join} {_itinerary_sections_join} {_itinerary_legs_join} WHERE {on_event_} {on_leg_} {on_section_};"

        stages_df = self.db_manager.read_sql(q)

        # TO DO move this into the SQL query
        if stage_code:
            # Try to be flexible with stage codes as code or stageId
            if stage_code[0].startswith("SS"):
                stages_df = stages_df[stages_df["code"].isin(stage_code)]
            elif isinstance(stage_code[0], int):
                stages_df = stages_df[stages_df["stageId"].isin(stage_code)]

        return stages_df

    def getStageSplitPoints(self, stageId=None, raw=False, extended=False, updateDB=False):
        # TO DO - also support eventId?
        # TO DO also support raw
        if updateDB:
            self._getStages(updateDB=updateDB)
        if not stageId:
            q = "SELECT * FROM split_points;"
        else:
            q = f"SELECT * FROM split_points WHERE stageId={stageId};"

        stage_split_points_df = self.db_manager.read_sql(q)

        if raw and not extended:
            return stage_split_points_df

        # Optionally add in the final stage distance
        final_ =  len(stage_split_points_df)+1
        if extended:
            stage_info_ = self.getStageInfo(stage_code=stageId)
            if not stage_info_.empty:
                final_distance = stage_info_["distance"].iloc[0]
                final_row = DataFrame([{"stageId":stageId, "number":final_,"distance": final_distance}])
                stage_split_points_df = concat(
                [stage_split_points_df, final_row], ignore_index=True
            )

        stage_split_points_df = stage_split_points_df.sort_values("number")
        stage_split_points_df["distance_"] = (
                            stage_split_points_df["distance"]
                            .diff()
                            .fillna(stage_split_points_df["distance"])
                            .round(2)
                        )

        stage_split_points_df["name"] = self.SPLIT_PREFIX + stage_split_points_df["number"].astype(str)

        if extended:
            stage_split_points_df["name"] = stage_split_points_df["name"].replace(
                f"{self.SPLIT_PREFIX}{final_}", "FINAL"
            )

        return stage_split_points_df

    def getStageControls(self, updateDB=False, raw=True):
        if updateDB:
            self._getStages(updateDB=updateDB)

        if raw:
            q = "SELECT * FROM stage_controls;"
        # TO DO a query that gives a "pretty" result

        stage_controls_df = self.db_manager.read_sql(q)

        return stage_controls_df

    def _getEvent(self, *args, **kwargs):
        """This also sets self.rallyId, self.itineraryId"""
        kwargs["eventId"] = self.eventId

        eventData_df, eventRallies_df, eventClasses_df = self.api_client._getEvent(
            *args, **kwargs
        )

        _event_df = eventRallies_df[eventRallies_df["isMain"] == True].iloc[0]
        self.rallyId = int(_event_df["rallyId"])
        self.itineraryId = int(_event_df["itineraryId"])

        return eventData_df, eventRallies_df, eventClasses_df

    def _setEvent(self, r, updateDB=True):
        if not r.empty:
            self.eventId = int(r.iloc[0]["eventId"])
            self.eventName = r.iloc[0]["name"]
            # Get the event info
            self._getEvent(updateDB=updateDB)
            self._getStages(updateDB=updateDB)
            self._getEntries(updateDB=updateDB)
            self._getEventItineraries(updateDB=updateDB)

    def setEventById(self, eventId=None, updateDB=True):
        if not eventId:
            return
        # Do this as a fuzzy search?
        q = f'SELECT eventId, name FROM season_rounds WHERE eventId="{eventId}";'
        r = self.db_manager.read_sql(q)
        # HACK TO DO this is a fudge
        self._setEvent(r, updateDB)

    def setEventByName(self, name=None, updateDB=True):
        if not name:
            return
        # Do this as a fuzzy search?
        q = f'SELECT eventId, name FROM season_rounds WHERE name="{name}";'
        r = self.db_manager.read_sql(q)
        # HACK TO DO this is a fudge
        self._setEvent(r, updateDB)

    def setChampionship(self, championshipId=None):
        championships_df = self.getChampionships()
        self.championshipName = self._getChampionshipName()
        # TO DO - defend against brokenness here
        if championshipId:
            self.championshipId = championshipId
        else:
            self.championshipId = int(
                championships_df[championships_df["name"] == self.championshipName].iloc[0][
                    "championshipId"
                ]
            )
        if self.championshipId:
            self._getChampionshipOverallResults(updateDB=True)
            self._getChampionshipDetail(updateDB=True)

    def _setStage(self, r, updateDB=False):
        # HACK TO DO this is a fudge
        if not r.empty:
            r = r.iloc[0]
            self.stageId = int(r["stageId"])
            self.stageName = r["name"]
            self.stageCode = r["code"]
            # Get the event info
            self._getStages(updateDB=updateDB)

    def setStageById(self, stageId=None, updateDB=True):
        # TO DO - we need to clobber the update if the table is complete
        self._getStages(updateDB=updateDB)
        if not stageId:
            return
        # Do this as a fuzzy search?
        q = f'SELECT stageId, name, code FROM stage_info WHERE stageId="{int(stageId)}";'
        r = self.db_manager.read_sql(q)
        self._setStage(r, updateDB=updateDB)

    def setStageByCode(self, stageCode=None, updateDB=True):
        # TO DO - we need to clobber the update if the table is complete
        self._getStages(updateDB=updateDB)
        if not stageCode:
            return
        # Do this as a fuzzy search?
        q = f'SELECT stageId, name, code FROM stage_info WHERE code="{stageCode}";'
        r = self.db_manager.read_sql(q)
        self._setStage(r, updateDB=updateDB)

    # TO DO a way of setting self.controlId; also need strategies for invalidating Ids
    def _getControlTimes(self, *args, stageId=None, **kwargs):
        stageId = stageId if stageId else self.stageId
        kwargs["eventId"] = self.eventId
        kwargs["controlId"] = self.controlId

        return self.api_client._getControlTimes(*args, **kwargs)

    def _getStageTimes(self, *args, stageId=None, **kwargs):
        stageId = stageId if stageId else self.stageId
        kwargs["eventId"] = self.eventId
        kwargs["rallyId"] = self.rallyId
        kwargs["stageId"] = stageId

        return self.api_client._getStageTimes(*args, **kwargs)

    def getStageTimes(
        self,
        stageId=None,
        priority=None,
        rebaseToCategory=True,
        raw=True,
        updateDB=False,
    ):
        if updateDB:
            self._getStageTimes(stageId=stageId)

        stageId = stageId if stageId else self.stageId
        if stageId and self.eventId and self.rallyId:
            priority = None if priority == "P0" else priority
            on_event_ = f"AND st.eventId={self.eventId} AND st.stageId={stageId} AND st.rallyId={self.rallyId}"
            priority_ = f"""AND e.priority="{priority}" """ if priority else ""
            if raw:
                sql = f"""SELECT * FROM stage_times AS st WHERE 1=1 {on_event_} {priority_};"""
            else:
                _entry_join = f"INNER JOIN entries AS e ON st.entryId=e.entryId"
                _driver_join = (
                    f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
                )
                _codriver_join = (
                    f"INNER JOIN entries_codrivers AS cd ON e.codriverId=cd.personId"
                )
                _manufacturer_join = f"INNER JOIN manufacturers AS m ON e.manufacturerId=m.manufacturerId"
                _entrants_join = f"INNER JOIN entrants AS n ON e.entrantId=n.entrantId"
                sql = f"SELECT d.fullName AS driverName, cd.fullName AS codriverName, m.name AS manufacturerName, n.name AS entrantName, e.vehicleModel, e.identifier AS carNo, st.* FROM stage_times AS st {_entry_join} {_driver_join} {_codriver_join} {_manufacturer_join} {_entrants_join} WHERE 1=1 {on_event_} {priority_};"

            r = self.db_manager.read_sql(sql)
            # Hack to poll API if empty
            if r.empty:
                self._getStageTimes(stageId=stageId, updateDB=True)
                r = self.db_manager.read_sql(sql)
        else:
            r = DataFrame()

        if r.empty:
            print(f"No getStageTimes? {self.eventId} {self.stageId} {self.rallyId}")
            return r

        df_stageTimes = r

        df_stageTimes["roadPos"] = range(1, len(df_stageTimes) + 1)
        df_stageTimes.sort_values("position", inplace=True)
        df_stageTimes["categoryPosition"] = range(1, len(df_stageTimes) + 1)
        df_stageTimes.sort_values("roadPos", inplace=True)

        # TO DO  - we need to rebase times to category / categoryPositionLeader
        if rebaseToCategory:
            pass
            # TO DO - need to initialise things back to category leader
        if "pos" in df_stageTimes:
            df_stageTimes["pos"] = df_stageTimes["pos"].astype("Int64")

        if "diffFirst" in df_stageTimes:
            df_stageTimes["Gap"] = df_stageTimes["diffFirstMs"].apply(
                lambda x: round(x / 1000, 1)
            )
        if "diffPrev" in df_stageTimes:
            df_stageTimes["Diff"] = df_stageTimes["diffPrevMs"].apply(
                lambda x: round(x / 1000, 1)
            )
        if "elapsedDurationMs" in df_stageTimes:
            df_stageTimes["timeInS"] = df_stageTimes["elapsedDurationMs"].apply(
                lambda x: round(x / 1000, 1)
            )
            df_stageTimes["timeToCarBehind"] = (
                df_stageTimes["timeInS"].diff(-1).round(1)
            )
            # Pace annotations

            df_stageDetails = self.getStageInfo()
            stage_dist = float(
                df_stageDetails.loc[
                    df_stageDetails["stageId"] == stageId, "distance"
                ].iloc[0]
            )
            df_stageTimes["speed (km/h)"] = (
                stage_dist / (df_stageTimes["timeInS"] / 3600)
            ).round(1)
            # Use .loc[] to modify the original DataFrame in place
            df_stageTimes["pace (s/km)"] = (
                df_stageTimes["timeInS"] / stage_dist
            ).round(2)

            p1_ = df_stageTimes[df_stageTimes["categoryPosition"] == 1].iloc[0]
            df_stageTimes["pace diff (s/km)"] = (
                df_stageTimes["pace (s/km)"] - p1_["pace (s/km)"]
            ).round(2)
            # A percent diff is always relative to something
            # In rebasing, we need to work with the actual times
            # so handle percentage diffs in the display logic for now?
            # df_stageTimes["percent"] = 100 * df_stageTimes["timeInS"] / df_stageTimes.loc[0,"timeInS"]
        return df_stageTimes

    def _getSplitTimes(self, *args, stageId=None, **kwargs):
        kwargs["eventId"] = self.eventId
        kwargs["rallyId"] = self.rallyId
        kwargs["stageId"] = stageId

        return self.api_client._getSplitTimes(*args, **kwargs)

    def getSplitTimes(self, stageId=None, priority=None, raw=True, updateDB=False):
        if updateDB:
            self._getSplitTimes(stageId=stageId, updateDB=updateDB)
        stageId = stageId if stageId else self.stageId

        if stageId and self.eventId and self.rallyId:
            priority = None if priority == "P0" else priority
            on_event_ = f"AND spt.eventId={self.eventId} AND spt.stageId={stageId} AND spt.rallyId={self.rallyId}"
            priority_ = f"""AND e.priority="{priority}" """ if priority else ""
            split_points_join = (
                "INNER JOIN split_points AS spp ON spp.splitPointId=spt.splitPointId"
            )

            if raw:
                sql = f"""SELECT spt.*, spp.number FROM split_times AS spt {split_points_join} WHERE 1=1 {on_event_} {priority_};"""
            else:
                _entry_join = f"INNER JOIN entries AS e ON spt.entryId=e.entryId"
                _driver_join = (
                    f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
                )
                sql = f"SELECT d.fullName AS driverName, e.identifier as carNo, e.vehicleModel, spt.*, ROUND(spt.elapsedDurationMs/1000, 2) AS elapsedTimeInS, spp.number FROM split_times AS spt {split_points_join} {_entry_join} {_driver_join} WHERE 1=1 {on_event_} {priority_};"

            r = self.db_manager.read_sql(sql)
            # Hack to poll API if empty
            if r.empty:
                self._getSplitTimes(stageId=stageId, updateDB=True)
                r = self.db_manager.read_sql(sql)
        else:
            print(f"No getSplitTimes? {self.eventId} {self.stageId} {self.rallyId}")
            r = DataFrame()

        return r

    def getSplitTimesWide(self, stageId=None, priority=None, extended=False, timeInS=True, split_cols=None, updateDB=None):
        if updateDB:
            self._getSplitTimes(stageId=stageId, updateDB=updateDB)

        split_times_df = self.getSplitTimes(
                            stageId=stageId, priority=priority, raw=False
                        )

        if split_times_df.empty:
            return

        split_times_df["number"] = self.SPLIT_PREFIX + split_times_df[
            "number"
        ].astype(str)
        split_times_wide = pivot(
            split_times_df.dropna(
                subset=["number", "elapsedDurationMs"]
            ),
            index=["driverName", "entryId", "carNo"],
            columns="number",
            values="elapsedDurationMs",
        ).reset_index()

        # Optionally add in the final stage time
        if extended:
            stage_times = self.getStageTimes(stageId=stageId)[
                ["entryId", "elapsedDurationMs"]
            ]
            stage_times.rename(
                columns={"elapsedDurationMs": self.SPLIT_FINAL}, inplace=True
            )
            split_times_wide = merge(
                split_times_wide, stage_times, on="entryId"
            )

        if timeInS:
            split_cols = self.getSplitCols(split_times_wide) if not split_cols else split_cols
            split_times_wide[split_cols] = (split_times_wide[split_cols] / 1000).round(
                1
            )

        split_times_wide.drop(columns=["entryId"], inplace=True)

        return split_times_wide

    def getSplitCols(self, split_times_wide):
        """Get the split time columns from the wide splits dataframe."""
        split_cols = [c for c in split_times_wide.columns if c.startswith(self.SPLIT_PREFIX) or c == self.SPLIT_FINAL]
        return split_cols

    def getSplitDuration(self, split_times_wide, split_cols=None, ret_id=True, id_col=None):
        """The time it takes a car to traverse a split section (split_times_wide)."""
        id_col = "carNo" if not id_col else id_col
        id_col = [id_col] if isinstance(id_col, str) else id_col

        # Ensure split_cols are strings
        split_cols = self.getSplitCols(split_times_wide) if not split_cols else split_cols

        # Create a copy of the dataframe with selected columns
        df_ = split_times_wide[split_cols].copy()

        # Calculate differences between consecutive columns
        diff_df = df_[split_cols[1:]].values - df_[split_cols[:-1]].values

        # Convert back to dataframe
        diff_df = DataFrame(diff_df, columns=split_cols[1:], index=df_.index)

        # Add first split column back
        diff_df[split_cols[0]] = df_[split_cols[0]]

        if ret_id:
            # Add entryId column
            diff_df[id_col] = split_times_wide[id_col]

            # Reorder columns
            cols = id_col + split_cols
            return diff_df[cols]

        return diff_df

    def getScaledSplits(self, stageId, priority, view):

        split_times_wide = self.getSplitTimesWide(
                                stageId=stageId, priority=priority, extended=True, timeInS=True
                            )

        if split_times_wide.empty:
            return

        split_cols = self.getSplitCols(split_times_wide)

        if view in ["time_acc", "pos_acc"]:
            # TO DO  precision number format formatting
            # styles = {c: "{0:0.1f}" for c in split_cols}
            # return split_times_wide.style.format(styles)

            split_times_wide.loc[:, split_cols] = split_times_wide[
                split_cols
            ].round(1)

            if view == "pos_acc":
                split_times_wide.loc[:, split_cols] = split_times_wide[
                    split_cols
                ].rank(method="min", na_option="keep")

            # split_times_wide.columns = (
            #    ["Driver", "TeamName", "RoadPos"]
            #    + [f"Split {i}" for i in range(1, len(split_cols))]
            #    + ["Finish"]
            # )
            return split_times_wide

        split_durations = self.getSplitDuration(
                                split_times_wide, id_col=["carNo", "driverName"]
                            )

        split_dists_ = self.getStageSplitPoints(
                                stageId=stageId, extended=True
                            )
        split_dists = split_dists_.set_index("name")[
                                "distance_"
                            ].to_dict()

        if split_dists:
            scaled_splits_wide = split_durations.copy()
            if view == "pos_within":
                scaled_splits_wide.loc[:, split_cols] = scaled_splits_wide[split_cols].rank(
                    method="min", na_option="keep"
                )
            elif view == "pace":
                scaled_splits_wide.update(
                    scaled_splits_wide.loc[:, split_dists.keys()].apply(
                        lambda s: s / split_dists[s.name]
                    )
                )
            elif view == "speed":
                scaled_splits_wide.update(
                    scaled_splits_wide.loc[:, split_dists.keys()].apply(
                        lambda s: 3600 * split_dists[s.name] / s
                    )
                )

        # styles = {c: "{0:0.1f}" for c in split_cols}

        if not view.startswith("pos_"):
            scaled_splits_wide.loc[:, split_cols] = scaled_splits_wide[split_cols].round(1)

        return scaled_splits_wide

    def _getStageOverallResults(
        self, *args, stageId=None, by_championship=False, **kwargs
    ):
        kwargs["eventId"] = self.eventId
        kwargs["rallyId"] = self.rallyId
        kwargs["championshipId"] = self.championshipId
        kwargs["stageId"] = stageId
        kwargs["by_championship"] = by_championship

        return self.api_client._getStageOverallResults(*args, **kwargs)

    def getStageOverallResults(
        self, stageId=None, priority=None, raw=True, updateDB=False
    ):
        if updateDB:
            self._getStageOverallResults(stageId=stageId, updateDB=updateDB)

        stageId = stageId if stageId else self.stageId
        if self.eventId and stageId and self.rallyId:
            priority = None if priority == "P0" else priority
            _entry_join = f"INNER JOIN entries AS e ON o.entryId=e.entryId"
            on_event_ = f"AND o.eventId={self.eventId} AND o.stageId={stageId} AND o.rallyId={self.rallyId}"

            priority_ = f"""AND e.priority="{priority}" """ if priority else ""

            if raw:
                sql = f"""SELECT * FROM stage_overall AS o {_entry_join} WHERE 1=1 {on_event_} {priority_};"""
            else:
                _entry_join = f"INNER JOIN entries AS e ON o.entryId=e.entryId"
                _driver_join = (
                    f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
                )
                _codriver_join = (
                    f"INNER JOIN entries_codrivers AS cd ON e.codriverId=cd.personId"
                )
                _manufacturer_join = f"INNER JOIN manufacturers AS m ON e.manufacturerId=m.manufacturerId"
                _entrants_join = f"INNER JOIN entrants AS n ON e.entrantId=n.entrantId"
                sql = f"SELECT d.fullName AS driverName, e.vehicleModel, e.identifier AS carNo, cd.fullName AS codriverName, m.name AS manufacturerName, n.name AS entrantName, e.priority, e.eligibility, o.* FROM stage_overall AS o {_entry_join} {_driver_join} {_codriver_join} {_manufacturer_join} {_entrants_join} WHERE 1=1 {on_event_} {priority_};"
            r = self.db_manager.read_sql(sql)
            # Hack to poll API if empty
            if r.empty:
                self._getStageOverallResults(stageId=stageId, updateDB=True)
                r = self.db_manager.read_sql(sql)
        else:
            print(
                f"No getStageOverallResults? {self.eventId} {self.stageId} {self.rallyId}"
            )
            r = DataFrame()
        overall_df = r
        overall_df["roadPos"] = range(1, len(overall_df) + 1)
        overall_df.sort_values("position", inplace=True)
        overall_df["categoryPosition"] = range(1, len(overall_df) + 1)
        overall_df.sort_values("roadPos", inplace=True)
        return r

    def _getStageWinners(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        kwargs["rallyId"] = self.rallyId
        return self.api_client._getStageWinners(*args, **kwargs)

    def _getRetirements(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        return self.api_client._getRetirements(*args, **kwargs)

    def _getPenalties(self, *args, **kwargs):
        kwargs["eventId"] = self.eventId
        return self.api_client._getPenalties(*args, **kwargs)

    def getStageWinners(self, on_event=True, priority=None, raw=True):
        if not self.eventId or not self.rallyId:
            return DataFrame()

        # TO DO - handle priority; maybe create a category_stagewinners db table?
        _on_event = (
            f"""w.eventId={self.eventId} AND w.rallyId={self.rallyId}"""
            if on_event
            else ""
        )

        if raw:
            sql = f"""SELECT * FROM stagewinners AS w WHERE {_on_event};"""
        else:
            _entry_join = f"INNER JOIN entries AS e ON w.entryId=e.entryId"
            _stages_join = f"INNER JOIN stage_info AS st ON st.stageId=w.stageId"
            _driver_join = f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
            _codriver_join = (
                f"INNER JOIN entries_codrivers AS cd ON e.codriverId=cd.personId"
            )
            _manufacturer_join = (
                f"INNER JOIN manufacturers AS m ON e.manufacturerId=m.manufacturerId"
            )
            _entrants_join = f"INNER JOIN entrants AS n ON e.entrantId=n.entrantId"
            _itinerary_stages_join = (
                f"INNER JOIN itinerary_stages AS it_st ON it_st.stageId=w.stageId"
            )
            _itinerary_sections_join = f"INNER JOIN itinerary_sections AS it_se ON it_se.itinerarySectionId=it_st.itinerarySectionId"
            _itinerary_legs_join = f"INNER JOIN itinerary_legs AS it_l ON it_l.itineraryLegId=it_st.itineraryLegId"
            sql = f"""SELECT d.fullName AS driverName, cd.fullName AS codriverName, m.name AS manufacturerName, n.name AS entrantName, e.identifier AS carNo, e.vehicleModel, it_st.code, it_se.name AS sectionName, it_l.name AS day, st.distance, w.* FROM stagewinners AS w {_entry_join} {_stages_join} {_driver_join} {_codriver_join} {_manufacturer_join} {_entrants_join} {_itinerary_stages_join} {_itinerary_sections_join} {_itinerary_legs_join} WHERE {_on_event};"""

        r = self.db_manager.read_sql(sql)
        # Hack to poll API if empty
        if r.empty:
            self._getStageWinners(updateDB=True)
            r = self.db_manager.read_sql(sql)
        return r

    # TO DO - offer more search limits
    def getRetirements(self, on_event=True, raw=True, updateDB=False):
        if updateDB:
            self._getRetirements(updateDB=updateDB)

        _on_event = f"r.eventId={self.eventId}" if on_event and self.eventId else ""

        if raw:
            _on_event = f"WHERE {_on_event}" if _on_event else _on_event
            sql = f"""SELECT * FROM retirements r {_on_event};"""
        else:
            # Need to merge entryId and controlId
            _entry_join = f"INNER JOIN entries AS e ON r.entryId=e.entryId"
            _driver_join = f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
            _codriver_join = (
                f"INNER JOIN entries_codrivers AS cd ON e.codriverId=cd.personId"
            )
            _manufacturer_join = (
                f"INNER JOIN manufacturers AS m ON e.manufacturerId=m.manufacturerId"
            )
            _entrants_join = f"INNER JOIN entrants AS n ON e.entrantId=n.entrantId"
            _control_join = f"INNER JOIN stage_controls AS c ON r.controlId=c.controlId"
            _on_event = f"WHERE {_on_event}" if _on_event else _on_event
            sql = f"""SELECT d.fullName AS driverName, cd.fullName AS codriverName, m.name AS manufacturerName, n.name AS entrantName, e.identifier AS carNo, e.vehicleModel, c.code, r.reason, c.location, c.type, r.retirementDateTime, r.status FROM retirements r {_entry_join} {_driver_join} {_codriver_join} {_manufacturer_join} {_entrants_join} {_control_join} {_on_event};"""

        r = self.db_manager.read_sql(sql)
        # Hack to poll API if empty
        if r.empty:
            self._getRetirements(updateDB=True)
            r = self.db_manager.read_sql(sql)
        return r

    # TO DO - offer more search limits
    def getPenalties(self, on_event=True, raw=True, updateDB=False):
        if updateDB:
            self._getPenalties(updateDB=updateDB)

        _on_event = f"p.eventId={self.eventId}" if on_event and self.eventId else ""

        if raw:
            _on_event = f"WHERE {_on_event}" if _on_event else _on_event
            sql = f"""SELECT * FROM penalties AS p {_on_event};"""
        else:
            # Need to merge entryId and controlId
            _entry_join = f"INNER JOIN entries AS e ON p.entryId=e.entryId"
            _driver_join = f"INNER JOIN entries_drivers AS d ON e.driverId=d.personId"
            _codriver_join = (
                f"INNER JOIN entries_codrivers AS cd ON e.codriverId=cd.personId"
            )
            _manufacturer_join = (
                f"INNER JOIN manufacturers AS m ON e.manufacturerId=m.manufacturerId"
            )
            _entrants_join = f"INNER JOIN entrants AS n ON e.entrantId=n.entrantId"
            _control_join = f"INNER JOIN stage_controls AS c ON p.controlId=c.controlId"
            _on_event = f"WHERE {_on_event}" if _on_event else _on_event
            sql = f"""SELECT d.fullName AS driverName, cd.fullName AS codriverName, m.name AS manufacturerName, n.name AS entrantName, e.identifier AS carNo, e.vehicleModel, c.code, p.penaltyDuration, p.Reason, c.location, c.type FROM penalties AS p {_entry_join} {_driver_join} {_codriver_join} {_manufacturer_join} {_entrants_join} {_control_join} {_on_event};"""

        r = self.db_manager.read_sql(sql)
        # Hack to poll API if empty
        if r.empty:
            self._getPenalties(updateDB=True)
            r = self.db_manager.read_sql(sql)
        return r

    def query(self, sql):
        r = self.db_manager.read_sql(sql)
        return r

## TO DO - update for use with ERC

- rounds load ok
- TO DO - clear, identify and load categories
- TO DO - clear and load days
- TO DO - clear and load sections
- TO DO - clear and load stages
- TO DO - items around _getChampionshipName need clarifying; only for championship results? Not in-rally?


In [2]:
wrc = WRCTimingResultsAPIClientV2(newDB=True)

INFO:__main__:Initialising the database...
INFO:__main__:Creating new db tables...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/seasons.json


INFO:__main__:Updating seasons...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/season-detail.json?seasonId=34


INFO:__main__:Updating championship_lookup...
INFO:__main__:Updating season_rounds...


In [3]:
wrc.getSeasons()

,seasonId,name,year
0,1,World Rally Championship,2018
1,4,World Rally Championship,2019
2,6,World Rally Championship,2020
3,8,World Rally Championship,2021
4,11,World Rally Championship,2022
5,12,European Rally Championship,2022
6,20,World Rally Championship,2023
7,21,European Rally Championship,2023
8,28,World Rally Championship,2024
9,29,European Rally Championship,2024


In [95]:
# From the seasons we can search on year and name
# Names: "World Rally Championship", "European Rally Championship"
# Return: seasonId (e.g World Rally Championship 2025: seasonId=34)
#
wrc.getSeasonRounds()

,categories,clerkOfTheCourse,country.countryId,country.iso2,country.iso3,country.name,countryId,eventId,finishDate,location,...,startDate,stewards,surfaces,templateFilename,timeZoneId,timeZoneName,timeZoneOffset,trackingEventId,order,seasonId
0,None,Romain PUGLIESE,147,MC,MCO,Monaco,147,534,2025-01-26,"Gap, France",...,2025-01-23,Tanja GEILHAUSEN Edoardo DELLEANI ...,Mixed,results-report-templates/e352213e-0a5d-443f-b9...,Europe/Monaco,(UTC+01:00) Central European Time (Monaco),60,4208,1,34
1,None,Stig Rune Kjernsli,215,SE,SWE,Sweden,215,535,2025-02-16,"Umeå, Västerbotten County",...,2025-02-13,Mazen Al-Hilli Mathieu Remmerie ...,Snow,results-report-templates/99d0742c-4b8a-4bc9-ab...,Europe/Stockholm,(UTC+01:00) Central European Time (Stockholm),60,4209,2,34
2,None,George Mwangi,116,KE,KEN,Kenya,116,536,2025-03-23,Nairobi,...,2025-03-20,Tanja Geilhausen Alice Scutellà ...,Gravel,results-report-templates/fde0030f-74a1-48a6-ba...,Africa/Nairobi,(UTC+03:00) East Africa Time (Nairobi),180,4210,3,34
3,None,None,209,ES,ESP,Spain,209,538,2025-04-27,"Las Palmas, Canarias",...,2025-04-24,None,Tarmac,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4211,4,34
4,None,None,178,PT,PRT,Portugal,178,540,2025-05-18,"Matosinhos, Porto",...,2025-05-15,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4212,5,34
5,None,None,110,IT,ITA,Italy,110,542,2025-06-08,"Alghero, Sardinia",...,2025-06-05,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4213,6,34
6,None,None,86,GR,GRC,Greece,86,544,2025-06-29,"Lamia, Central Greece",...,2025-06-26,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4214,7,34
7,None,None,70,EE,EST,Estonia,70,546,2025-07-20,Tartu,...,2025-07-17,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4215,8,34
8,None,None,75,FI,FIN,Finland,75,547,2025-08-03,Jyväskylä,...,2025-07-31,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4216,9,34
9,None,None,173,PY,PRY,Paraguay,173,549,2025-08-31,Itapúa,...,2025-08-28,None,Gravel,None,AUS Eastern Standard Time,(UTC+10:00) Eastern Australia Time (Sydney) (...,660,4217,10,34


In [4]:
wrc.setEventByName("Rally Sweden")
wrc.eventId, wrc.eventName, wrc.year

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535.json


INFO:__main__:Updating event_classes...
INFO:__main__:Updating event_rallies...
INFO:__main__:Updating event_date...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages.json


INFO:__main__:Updating stage_info...
INFO:__main__:Updating split_points...
INFO:__main__:Updating stage_controls...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/entries.json


...

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/startLists/1813.json


INFO:__main__:Updating startlists...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/startLists/1815.json


INFO:__main__:Updating startlists...


(535, 'Rally Sweden', 2025)

In [97]:
wrc.getItineraryLegs()

,itineraryId,itineraryLegId,eventId,legDate,name,order,startListId,status
0,1137,1506,535,2025-02-13,Thursday 13th February,1,1810,Completed
1,1137,1507,535,2025-02-14,Friday 14th February,2,1811,Completed
2,1137,1508,535,2025-02-15,Saturday 15th February,3,1813,Completed
3,1137,1509,535,2025-02-16,Sunday 16th February,4,1815,Completed


In [98]:
wrc._getStartLists(startListId=1810)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/startLists/1810.json


,startListItemId,startListId,entryId,startDateTime,startDateTimeLocal,order,eventId,name
0,83434,1810,54778,2025-02-13T17:40:00,2025-02-13T18:40:00+01:00,1,535,Thursday
1,83435,1810,54780,2025-02-13T17:43:00,2025-02-13T18:43:00+01:00,2,535,Thursday
2,83436,1810,54782,2025-02-13T17:46:00,2025-02-13T18:46:00+01:00,3,535,Thursday
3,83437,1810,54779,2025-02-13T17:49:00,2025-02-13T18:49:00+01:00,4,535,Thursday
4,83438,1810,54777,2025-02-13T17:52:00,2025-02-13T18:52:00+01:00,5,535,Thursday
5,83439,1810,54786,2025-02-13T17:55:00,2025-02-13T18:55:00+01:00,6,535,Thursday
6,83440,1810,54781,2025-02-13T17:58:00,2025-02-13T18:58:00+01:00,7,535,Thursday
7,83441,1810,54783,2025-02-13T18:01:00,2025-02-13T19:01:00+01:00,8,535,Thursday
8,83442,1810,54784,2025-02-13T18:04:00,2025-02-13T19:04:00+01:00,9,535,Thursday
9,83443,1810,54785,2025-02-13T18:07:00,2025-02-13T19:07:00+01:00,10,535,Thursday


In [99]:
wrc.getItinerarySections()

,eventId,itineraryLegId,itinerarySectionId,name,order
0,None,1506,3635,Section 1,1
1,None,1507,3636,Section 2,2
2,None,1507,3637,Section 3,3
3,None,1507,3638,Section 4,4
4,None,1508,3639,Section 5,5
5,None,1508,3640,Section 6,6
6,None,1508,3641,Section 7,7
7,None,1509,3642,Section 8,8
8,None,1509,3643,Section 9,9
9,None,1509,3644,Section 10,10


In [100]:
wrc.getItineraryStages()

,itinerarySectionId,itineraryLegId,order,name,name_,stageId,eventId,number,distance,status,stageType,timingPrecision,locked,code
0,3635,1506,1,Umeå Sprint 1,Section 1,8330,535,1,5.16,Completed,SpecialStage,Tenth,1,SS1
1,3636,1507,2,Bygdsiljum 1,Section 2,8331,535,2,28.27,Completed,SpecialStage,Tenth,1,SS2
2,3636,1507,2,Andersvattnet 1,Section 2,8332,535,3,20.51,Completed,SpecialStage,Tenth,1,SS3
3,3636,1507,2,Bäck 1,Section 2,8333,535,4,10.80,Completed,SpecialStage,Tenth,1,SS4
4,3637,1507,3,Bygdsiljum 2,Section 3,8334,535,5,28.27,Completed,SpecialStage,Tenth,1,SS5
5,3637,1507,3,Andersvattnet 2,Section 3,8335,535,6,20.51,Completed,SpecialStage,Tenth,1,SS6
6,3637,1507,3,Bäck 2,Section 3,8336,535,7,10.80,Completed,SpecialStage,Tenth,1,SS7
7,3638,1507,4,Umeå Sprint 2,Section 4,8337,535,8,5.16,Completed,SuperSpecialStage,Tenth,1,SS8
8,3639,1508,5,Vännäs 1,Section 5,8338,535,9,15.65,Completed,SpecialStage,Tenth,1,SS9
9,3639,1508,5,Sarsjöliden 1,Section 5,8339,535,10,14.23,Completed,SpecialStage,Tenth,1,SS10


In [101]:
wrc.getItineraryControls()

,itinerarySectionId,itineraryLegId,order,name,controlId,eventId,stageId,type,code,location,...,targetDuration,targetDurationMs,firstCarDueDateTime,firstCarDueDateTimeLocal,status,controlPenalties,roundingPolicy,locked,bogey,bogeyMs
0,3635,1506,1,Section 1,35201,535,NaN,TimeControl,TC0,Podium Red Barn Arena,...,None,NaN,2025-02-13T17:40:00,2025-02-13T18:40:00+01:00,Completed,All,NoRounding,1,None,None
1,3635,1506,1,Section 1,35202,535,NaN,TimeControl,TC1A,Parc Fermé Umeå IN,...,00:30:00,1800000.0,2025-02-13T18:35:00,2025-02-13T19:35:00+01:00,Completed,Late,NoRounding,1,None,None
2,3636,1507,2,Section 2,35203,535,NaN,TimeControl,TC1B,Parc Fermé Umeå OUT,...,None,NaN,2025-02-14T07:05:00,2025-02-14T08:05:00+01:00,Completed,All,NoRounding,1,None,None
3,3636,1507,2,Section 2,35204,535,NaN,RegroupIn,TC4A,Regroup and Technical Zone IN,...,01:08:00,4080000.0,2025-02-14T11:35:00,2025-02-14T12:35:00+01:00,Completed,All,NoRounding,1,None,None
4,3637,1507,3,Section 3,35205,535,NaN,RegroupOut,TC4B,Regroup OUT / Service IN,...,00:20:00,1200000.0,2025-02-14T11:55:00,2025-02-14T12:55:00+01:00,Completed,None,NoRounding,1,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,3643,1509,9,Section 9,35277,535,8346.0,StageStart,SS17,Västervik 2,...,00:03:00,180000.0,2025-02-16T08:57:00,2025-02-16T09:57:00+01:00,Completed,None,RoundToClosestMinute,1,None,None
77,3643,1509,9,Section 9,35278,535,8346.0,FlyingFinish,SF17,Västervik 2,...,None,NaN,None,None,Completed,None,NoRounding,1,None,None
78,3644,1509,10,Section 10,35279,535,8347.0,TimeControl,TC18,Umeå,...,00:17:00,1020000.0,2025-02-16T11:12:00,2025-02-16T12:12:00+01:00,Completed,All,NoRounding,1,None,None
79,3644,1509,10,Section 10,35280,535,8347.0,StageStart,SS18,Umeå 2 (Power Stage),...,00:03:00,180000.0,2025-02-16T11:15:00,2025-02-16T12:15:00+01:00,Interrupted,None,RoundToClosestMinute,1,None,None


In [102]:
wrc.getEntries()
# TO DO: WRC is priority=P1, then for the championship "M" in eligibility
# TO DO: WRC2 etc is priority=P2 and "WRC2" in eleigibility; similarly WRC3, JWRC
# TO DO  DM, CM ?

,entryId,eventId,rallyId,driverId,codriverId,manufacturerId,entrantId,groupId,identifier,vehicleModel,entryListOrder,pbf,drive,eligibility,priority,status,tyreManufacturer
0,54777,535,583,762,851,33,166,152,1,i20 N Rally1,1,None,None,M,P1,Entry,None
1,54778,535,583,534,553,84,91,152,33,GR Yaris Rally1,2,None,None,M,P1,Entry,None
2,54779,535,583,524,525,33,166,152,8,i20 N Rally1,3,None,None,M,P1,Entry,None
3,54780,535,583,4676,4945,33,166,152,16,i20 N Rally1,4,None,None,M,P1,Rejoined,None
4,54781,535,583,785,5507,84,91,152,18,GR Yaris Rally1,5,None,None,M,P1,Entry,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,54834,535,583,14234,17360,33,1148,98,70,i20 N,58,None,None,(DM),,Retired,None
58,54835,535,583,17361,17362,77,3801,98,71,Fabia Evo,59,None,None,,,Entry,None
59,54836,535,583,5525,17363,33,1380,98,72,NG i20,60,None,None,,,Entry,None
60,54837,535,583,610,611,77,1379,98,73,Fabia,61,None,None,,,Entry,None


In [103]:
wrc.getDrivers()

,personId,countryId,state,firstName,lastName,abbvName,fullName,code,licenseNumber,country.countryId,country.name,country.iso2,country.iso3
0,762,22,None,Thierry,NEUVILLE,T. NEUVILLE,Thierry NEUVILLE,NEU,None,22,Belgium,BE,BEL
1,534,235,None,Elfyn,EVANS,E. EVANS,Elfyn EVANS,EVA,None,235,United Kingdom,GB,GBR
2,524,70,None,Ott,TÄNAK,O. TÄNAK,Ott TÄNAK,TÄN,None,70,Estonia,EE,EST
3,4676,76,None,Adrien,FOURMAUX,A. FOURMAUX,Adrien FOURMAUX,FOU,None,76,France,FR,FRA
4,785,112,None,Takamoto,KATSUTA,T. KATSUTA,Takamoto KATSUTA,KAT,None,112,Japan,JP,JPN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,14234,174,None,Jorge,MARTÍNEZ,J. MARTÍNEZ,Jorge MARTÍNEZ,MAR,None,174,Peru,PE,PER
58,17361,76,,Didier,THORAL,D. THORAL,Didier THORAL,THO,None,76,France,FR,FRA
59,5525,76,None,Tony,RIBAUDO,T. RIBAUDO,Tony RIBAUDO,RIB,None,76,France,FR,FRA
60,610,76,None,Thibaut,POIZOT,T. POIZOT,Thibaut POIZOT,POI,None,76,France,FR,FRA


In [60]:
wrc.getCoDrivers()

,personId,countryId,state,firstName,lastName,abbvName,fullName,code,licenseNumber,country.countryId,country.name,country.iso2,country.iso3
0,851,22,None,Martijn,WYDAEGHE,M. WYDAEGHE,Martijn WYDAEGHE,WYD,None,22,Belgium,BE,BEL
1,553,235,None,Scott,MARTIN,S. MARTIN,Scott MARTIN,MAR,None,235,United Kingdom,GB,GBR
2,525,70,None,Martin,JÄRVEOJA,M. JÄRVEOJA,Martin JÄRVEOJA,JAR,None,70,Estonia,EE,EST
3,4945,76,None,Alexandre,CORIA,A. CORIA,Alexandre CORIA,COR,None,76,France,FR,FRA
4,5507,107,None,Aaron,JOHNSTON,A. JOHNSTON,Aaron JOHNSTON,JOH,None,107,Ireland,IE,IRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,17360,318,,José Alberto,AROS,J. AROS,José Alberto AROS,ARO,None,318,Chile IOC,CL,CHI
58,17362,76,,Olivier,BROUZE,O. BROUZE,Olivier BROUZE,BRO,None,76,France,FR,FRA
59,17363,76,,Cyrielle,DELORME,C. DELORME,Cyrielle DELORME,DEL,None,76,France,FR,FRA
60,611,76,None,Marion,GRAND,M. GRAND,Marion GRAND,GRA,None,76,France,FR,FRA


In [62]:
wrc.getManufacturers()

,manufacturerId,name,logoFilename
0,33,Hyundai,hyundai
1,84,Toyota,toyota
2,26,Ford,ford
3,77,Skoda,skoda
4,13,Citroen,citroen
5,69,Renault,renault


In [63]:
wrc.getEntrants()

,entrantId,name
0,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM
1,91,TOYOTA GAZOO RACING WRT
2,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM
3,166,HYUNDAI SHELL MOBIS WORLD RALLY TEAM
4,91,TOYOTA GAZOO RACING WRT
...,...,...
57,1148,JORGE MARTÍNEZ
58,3801,DIDIER THORAL
59,1380,TONY RIBAUDO
60,1379,THIBAUT POIZOT


In [35]:
_, _, _, _, _, entryGroups_df, eventClasses_df = wrc._getEntries()
eventClasses_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/entries.json


,eventClassId,eventId,name
0,2912,535,RC1
12,2913,535,RC2
33,2914,535,RC3
61,2915,535,RC5


In [36]:
eventData_df, eventRallies_df, eventClasses_df = wrc._getEvent()
eventData_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535.json


,eventId,countryId,eventProfileId,name,slug,location,startDate,finishDate,timeZoneId,timeZoneName,...,public,applyJumpStarts,displayFiaColumns,roundShakedownTimes,shakedownCount,displayLicenseNumber,country.countryId,country.name,country.iso2,country.iso3
0,535,215,2,Rally Sweden,2025-wrc-sweden,"Umeå, Västerbotten County",2025-02-13,2025-02-16,Europe/Stockholm,(UTC+01:00) Central European Time (Stockholm),...,False,True,True,True,1,False,215,Sweden,SE,SWE


In [37]:
wrc.getChampionships()

,championshipId,name,seasonId,type,fieldOneDescription,fieldTwoDescription,fieldThreeDescription,fieldFiveDescription,fieldFourDescription
0,287,FIA World Rally Championship for Drivers,34,Drivers,FirstName,LastName,CountryISO3,ManufacturerFull,Manufacturer
1,288,FIA World Rally Championship for Co-Drivers,34,Codrivers,FirstName,LastName,CountryISO3,TyreManufacturer,Manufacturer
2,289,FIA World Rally Championship for Manufacturers,34,Manufacturer,Name,Manufacturer,LogoFileName,,
3,290,FIA World Rally Championship for Teams,34,Manufacturer,Name,Manufacturer,LogoFileName,,
4,291,FIA WRC2 Championship for Drivers,34,DriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
5,292,FIA WRC2 Championship for Co-Drivers,34,CodriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
6,293,FIA WRC2 Challenger Championship for Drivers,34,DriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
7,294,FIA WRC2 Challenger Championship for Co-Drivers,34,CodriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer
8,295,FIA WRC2 Championship for Teams,34,Manufacturer,Name,Manufacturer,LogoFileName,,
9,296,FIA WRC Masters Cup for Drivers,34,DriversWithEntrant,FirstName,LastName,CountryISO3,Entrant,Manufacturer


In [38]:
wrc.setChampionship()
wrc.championshipId, wrc.championshipName

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/championship-overall-results.json?championshipId=287&seasonId=34


INFO:__main__:Updating championship_overall...
INFO:__main__:Updating championship_results...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/championship-detail.json?championshipId=287&seasonId=34


INFO:__main__:Updating championship_rounds_detail...
INFO:__main__:Updating championship_countries...
INFO:__main__:Updating championship_entries...


(287, 'FIA World Rally Championship for Drivers')

In [39]:
wrc.getDrivers(by_championship=True)

,personId,countryId,state,firstName,lastName,abbvName,fullName,code,licenseNumber,country.countryId,country.name,country.iso2,country.iso3
0,762,22,None,Thierry,NEUVILLE,T. NEUVILLE,Thierry NEUVILLE,NEU,None,22,Belgium,BE,BEL
1,534,235,None,Elfyn,EVANS,E. EVANS,Elfyn EVANS,EVA,None,235,United Kingdom,GB,GBR
2,524,70,None,Ott,TÄNAK,O. TÄNAK,Ott TÄNAK,TÄN,None,70,Estonia,EE,EST
3,4676,76,None,Adrien,FOURMAUX,A. FOURMAUX,Adrien FOURMAUX,FOU,None,76,France,FR,FRA
4,785,112,None,Takamoto,KATSUTA,T. KATSUTA,Takamoto KATSUTA,KAT,None,112,Japan,JP,JPN
5,700,75,None,Kalle,ROVANPERÄ,K. ROVANPERÄ,Kalle ROVANPERÄ,ROV,None,75,Finland,FI,FIN
6,5279,130,None,Grégoire,MUNSTER,G. MUNSTER,Grégoire MUNSTER,MUN,None,130,Luxembourg,LU,LUX
7,5280,75,None,Sami,PAJARI,S. PAJARI,Sami PAJARI,PAJ,None,75,Finland,FI,FIN
8,3169,301,None,Mārtinš,SESKS,M. SESKS,Mārtinš SESKS,SES,None,301,Latvia IOC,LV,LAT
9,5791,107,None,Joshua,MCERLEAN,J. MCERLEAN,Joshua MCERLEAN,MCE,None,107,Ireland,IE,IRL


In [40]:
wrc.getChampionshipOverall()

,championshipEntryId,overallPosition,overallPoints
0,6700,1,88
1,6702,2,52
2,6697,3,49
3,6699,4,33
4,6701,5,31
5,6698,6,31
6,6784,7,25
7,6782,8,19
8,6783,9,16
9,6836,10,8


In [41]:
wrc.getChampionshipByRound()

,championshipEntryId,championshipId,dropped,eventId,pointsBreakdown,position,publishedStatus,status,totalPoints,entryId
0,6700,287,0,535,25 + 5 + 5,1,Published,Finished,35,None
1,6700,287,0,536,25 + 2,1,Published,Finished,27,None
2,6700,287,0,534,17 + 5 + 4,2,Published,Finished,26,None
3,6702,287,0,536,15 + 4 + 4,3,Published,Finished,23,None
4,6702,287,0,535,15 + 2 + 3,3,Published,Finished,20,None
...,...,...,...,...,...,...,...,...,...,...
58,6706,287,0,536,-,-,Published,DidNotEnter,0,None
59,6706,287,0,535,-,-,Published,DidNotEnter,0,None
60,6840,287,0,536,-,-,Published,DidNotEnter,0,None
61,6840,287,0,534,-,-,Published,DidNotEnter,0,None


In [42]:
wrc.getChampionShipRounds()

,eventId,countryId,eventProfileId,country,name,slug,location,startDate,finishDate,year,...,clerkOfTheCourse,stewards,templateFilename,locked,public,applyJumpStarts,displayFiaColumns,roundShakedownTimes,shakedownCount,displayLicenseNumber
0,534,147,2,None,Rallye Monte-Carlo,2025-wrc-monte,"Gap, France",2025-01-23,2025-01-26,2025,...,Romain PUGLIESE,Tanja GEILHAUSEN Edoardo DELLEANI ...,results-report-templates/e352213e-0a5d-443f-b9...,0,0,0,0,0,0,0
1,535,215,2,None,Rally Sweden,2025-wrc-sweden,"Umeå, Västerbotten County",2025-02-13,2025-02-16,2025,...,Stig Rune Kjernsli,Mazen Al-Hilli Mathieu Remmerie ...,results-report-templates/99d0742c-4b8a-4bc9-ab...,0,0,0,0,0,0,0
2,536,116,2,None,Safari Rally Kenya,2025-wrc-kenya,Nairobi,2025-03-20,2025-03-23,2025,...,George Mwangi,Tanja Geilhausen Alice Scutellà ...,results-report-templates/fde0030f-74a1-48a6-ba...,0,0,0,0,0,0,0
3,538,209,2,None,Rally Islas Canarias,2025-wrc-canarias,"Las Palmas, Canarias",2025-04-24,2025-04-27,2025,...,None,None,None,0,0,0,0,0,0,0
4,540,178,2,None,Vodafone Rally de Portugal,2025-wrc-portugal,"Matosinhos, Porto",2025-05-15,2025-05-18,2025,...,None,None,None,0,0,0,0,0,0,0
5,542,110,2,None,Rally Italia Sardegna,2025-wrc-italy,"Alghero, Sardinia",2025-06-05,2025-06-08,2025,...,None,None,None,0,0,0,0,0,0,0
6,544,86,2,None,EKO Acropolis Rally Greece,2025-wrc-greece,"Lamia, Central Greece",2025-06-26,2025-06-29,2025,...,None,None,None,0,0,0,0,0,0,0
7,546,70,2,None,Delfi Rally Estonia,2025-wrc-estonia,Tartu,2025-07-17,2025-07-20,2025,...,None,None,None,0,0,0,0,0,0,0
8,547,75,2,None,Secto Rally Finland,2025-wrc-finland,Jyväskylä,2025-07-31,2025-08-03,2025,...,None,None,None,0,0,0,0,0,0,0
9,549,173,2,None,Rally del Paraguay,2025-wrc-paraguay,Itapúa,2025-08-28,2025-08-31,2025,...,None,None,None,0,0,0,0,0,0,0


In [97]:
wrc.getChampionshipEntries()

,championshipEntryId,championshipId,personId,entrantId,manufacturerId,tyreManufacturerId,FirstName,LastName,CountryISO3,Manufacturer,TyreManufacturer
0,6697,287,524,None,33,None,Ott,TÄNAK,EST,Hyundai,None
1,6698,287,700,None,84,None,Kalle,ROVANPERÄ,FIN,Toyota,None
2,6699,287,670,None,84,None,Sébastien,OGIER,FRA,Toyota,None
3,6700,287,534,None,84,None,Elfyn,EVANS,GBR,Toyota,None
4,6701,287,4676,None,33,None,Adrien,FOURMAUX,FRA,Hyundai,None
5,6702,287,762,None,33,None,Thierry,NEUVILLE,BEL,Hyundai,None
6,6703,287,5791,None,26,None,Joshua,MCERLEAN,IRL,Ford,None
7,6704,287,4727,None,13,None,Yohan,ROSSEL,FRA,Citroen,None
8,6705,287,17305,None,77,None,Nikolay,GRYAZIN,BUL,Skoda,None
9,6706,287,4068,None,33,None,Eric,CAMILLI,FRA,Hyundai,None


In [98]:
wrc.getChampionshipCountries()

,countryId,name,iso2,iso3
0,45,Chile,CL,CHL
1,70,Estonia,EE,EST
2,75,Finland,FI,FIN
3,86,Greece,GR,GRC
4,110,Italy,IT,ITA
5,112,Japan,JP,JPN
6,116,Kenya,KE,KEN
7,147,Monaco,MC,MCO
8,173,Paraguay,PY,PRY
9,178,Portugal,PT,PRT


In [99]:
wrc.getPenalties(raw=False)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/penalties.json


INFO:__main__:Updating penalties...


,driverName,codriverName,manufacturerName,entrantName,carNo,vehicleModel,code,penaltyDuration,reason,location,type
0,Jorge MARTÍNEZ,José Alberto AROS,Hyundai,JORGE MARTÍNEZ,70,i20 N,TC10,PT1M,1 MIN EARLY,Sarsjöliden,TimeControl
1,Pontus TIDEMAND,Jørgen ERIKSEN,Skoda,PONTUS TIDEMAND,24,Fabia RS,SS14,PT10S,FALSE START,Kolksele 2,StageStart


In [100]:
wrc.getRetirements(raw=False)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/retirements.json


INFO:__main__:Updating retirements...


,driverName,codriverName,manufacturerName,entrantName,carNo,vehicleModel,code,reason,location,type,retirementDateTime,status
0,Joosep NÕGENE,Aleks LESK,Ford,LIGHTGREY,62,Fiesta Rally3,TC2,TECHNICAL,Bygdsiljum,TimeControl,2025-02-14T09:40:08.934,Temporary
1,Diego DOMINGUEZ,Rogelio PEÑATE,Ford,DIEGO DOMINGUEZ,57,Fiesta Rally3,SF5,MECHANICAL,Bygdsiljum 2,FlyingFinish,2025-02-14T16:11:59.017,Temporary
2,Hikaru KOGURE,Topi LUHTINEN,Toyota,TOYOTA GAZOO RACING WRT NG,31,GR Yaris,SF6,MECHANICAL,Andersvattnet 2,FlyingFinish,2025-02-14T16:16:38.611,Temporary
3,Bernhard TEN BRINKE,Tom WOODBURN,Skoda,BERNHARD TEN BRINKE,38,Fabia RS,SF7,OFF ROAD,Bäck 2,FlyingFinish,2025-02-14T17:30:39.094,Temporary
4,Jorge MARTÍNEZ,José Alberto AROS,Hyundai,JORGE MARTÍNEZ,70,i20 N,TC7,MECHANICAL,Bäck,TimeControl,2025-02-14T17:50:10.996,Temporary
5,Mattéo CHATILLON,Maxence CORNUAU,Renault,MATTÉO CHATILLON,44,Clio Rally3,SF8,OFF ROAD,Umeå Sprint 2,FlyingFinish,2025-02-14T19:46:52.423,Temporary
6,Ville VATANEN,Jarno OTTMAN,Renault,VILLE VATANEN,46,Clio Rally3,SF9,TECHNICAL,Vännäs 1,FlyingFinish,2025-02-15T09:52:22.322,Temporary
7,André MARTINEZ,Matías ARANGUREN,Ford,ANDRÉ MARTINEZ,53,Fiesta Rally3,SF9,MECHANICAL,Vännäs 1,FlyingFinish,2025-02-15T10:03:12.645,Temporary
8,Tuukka KAUPPINEN,Sebastian VIRTANEN,Toyota,TUUKKA KAUPPINEN,34,GR Yaris,TC11,MECHANICAL,Kolksele,TimeControl,2025-02-15T10:39:42.075,Temporary
9,Bernhard TEN BRINKE,Tom WOODBURN,Skoda,BERNHARD TEN BRINKE,38,Fabia RS,SF10,OFF ROAD,Sarsjöliden 1,FlyingFinish,2025-02-15T11:16:02.126,Temporary


In [101]:
wrc.getStageWinners()

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/stagewinners.json


INFO:__main__:Updating stagewinners...


,elapsedDuration,elapsedDurationMs,entryId,stageId,stageName,eventId,rallyId
0,00:03:21.6000000,201600,54778,8330,Umeå Sprint 1,535,583
1,00:13:58.8000000,838800,54778,8331,Bygdsiljum 1,535,583
2,00:10:43.8000000,643800,54780,8332,Andersvattnet 1,535,583
3,00:05:54.6000000,354600,54780,8333,Bäck 1,535,583
4,00:13:50.1000000,830100,54781,8334,Bygdsiljum 2,535,583
5,00:10:49.2000000,649200,54779,8335,Andersvattnet 2,535,583
6,00:06:02.5000000,362500,54777,8336,Bäck 2,535,583
7,00:03:29.9000000,209900,54778,8337,Umeå Sprint 2,535,583
8,00:09:00.5000000,540500,54782,8338,Vännäs 1,535,583
9,00:06:37.5000000,397500,54778,8339,Sarsjöliden 1,535,583


In [ ]:
wrc.getStageInfo()

,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,locked
0,SS1,5.16,535,Umeå Sprint 1,1,8330,SpecialStage,Completed,Tenth,1
1,SS2,28.27,535,Bygdsiljum 1,2,8331,SpecialStage,Completed,Tenth,1
2,SS3,20.51,535,Andersvattnet 1,3,8332,SpecialStage,Completed,Tenth,1
3,SS4,10.80,535,Bäck 1,4,8333,SpecialStage,Completed,Tenth,1
4,SS5,28.27,535,Bygdsiljum 2,5,8334,SpecialStage,Completed,Tenth,1
5,SS6,20.51,535,Andersvattnet 2,6,8335,SpecialStage,Completed,Tenth,1
6,SS7,10.80,535,Bäck 2,7,8336,SpecialStage,Completed,Tenth,1
7,SS8,5.16,535,Umeå Sprint 2,8,8337,SuperSpecialStage,Completed,Tenth,1
8,SS9,15.65,535,Vännäs 1,9,8338,SpecialStage,Completed,Tenth,1
9,SS10,14.23,535,Sarsjöliden 1,10,8339,SpecialStage,Completed,Tenth,1


In [103]:
wrc.getStageSplitPoints()

,splitPointId,stageId,number,distance
0,13602,8331,1,2.50
1,13603,8331,2,5.43
2,13604,8330,1,4.39
3,13605,8331,5,12.50
4,13606,8331,4,10.11
...,...,...,...,...
95,13699,8347,3,7.84
96,13700,8347,1,0.54
97,13701,8347,2,2.77
98,13704,8344,2,3.15


In [104]:
wrc.getStageControls()

,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,targetDuration,targetDurationMs,timingPrecision,type,locked,bogey,bogeyMs,roundingPolicy
0,TC1,35228,All,4.97,535,2025-02-13T18:00:00,None,Umeå Sprint,8330,Completed,00:20:00,1200000.0,Minute,TimeControl,1,None,None,NoRounding
1,SS1,35229,None,5.16,535,2025-02-13T18:05:00,None,Umeå Sprint 1,8330,Completed,00:05:00,300000.0,Minute,StageStart,1,None,None,RoundToClosestMinute
2,SF1,35230,None,NaN,535,None,None,Umeå Sprint 1,8330,Completed,None,NaN,Tenth,FlyingFinish,1,None,None,NoRounding
3,TC2,35231,All,61.13,535,2025-02-14T08:15:00,None,Bygdsiljum,8331,Completed,01:10:00,4200000.0,Minute,TimeControl,1,None,None,NoRounding
4,SS2,35232,None,28.27,535,2025-02-14T08:18:00,None,Bygdsiljum 1,8331,Completed,00:03:00,180000.0,Minute,StageStart,1,None,None,RoundToClosestMinute
5,SF2,35233,None,NaN,535,None,None,Bygdsiljum 1,8331,Completed,None,NaN,Tenth,FlyingFinish,1,None,None,NoRounding
6,TC3,35234,All,22.00,535,2025-02-14T09:16:00,None,Andersvattnet,8332,Completed,00:58:00,3480000.0,Minute,TimeControl,1,None,None,NoRounding
7,SS3,35235,None,20.51,535,2025-02-14T09:19:00,None,Andersvattnet 1,8332,Completed,00:03:00,180000.0,Minute,StageStart,1,None,None,RoundToClosestMinute
8,SF3,35236,None,NaN,535,None,None,Andersvattnet 1,8332,Completed,None,NaN,Tenth,FlyingFinish,1,None,None,NoRounding
9,TC4,35237,All,41.01,535,2025-02-14T10:24:00,None,Bäck,8333,Completed,01:05:00,3900000.0,Minute,TimeControl,1,None,None,NoRounding


In [120]:
wrc.setStageByCode("SS4")
wrc.stageId, wrc.stageName, wrc.stageCode

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages.json


INFO:__main__:Updating stage_info...
INFO:__main__:Updating split_points...
INFO:__main__:Updating stage_controls...
INFO:__main__:Updating stage_info...
INFO:__main__:Updating split_points...
INFO:__main__:Updating stage_controls...


https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages.json


(8333, 'Bäck 1', 'SS4')

In [121]:
wrc.getStageTimes()

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages/8333/stagetimes.json?rallyId=583


INFO:__main__:Updating stage_times...


,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,...,status,eventId,rallyId,Gap,Diff,timeInS,timeToCarBehind,speed (km/h),pace (s/km),pace diff (s/km)
0,00:00:03.9000000,3900,00:00:03.9000000,3900,00:05:58.5000000,358500,54778,2,Default,8333,...,Completed,535,583,3.9,3.9,358.5,3.9,108.5,33.2,0.0
1,00:00:00,0,00:00:00,0,00:05:54.6000000,354600,54780,1,Default,8333,...,Completed,535,583,0.0,0.0,354.6,-5.0,109.6,32.8,-0.4
2,00:00:05,5000,00:00:00.4000000,400,00:05:59.6000000,359600,54782,4,Default,8333,...,Completed,535,583,5.0,0.4,359.6,0.0,108.1,33.3,0.1
3,00:00:05,5000,00:00:00,0,00:05:59.6000000,359600,54779,5,Assessed,8333,...,Completed,535,583,5.0,0.0,359.6,-1.6,108.1,33.3,0.1
4,00:00:06.6000000,6600,00:00:01.6000000,1600,00:06:01.2000000,361200,54777,6,Default,8333,...,Completed,535,583,6.6,1.6,361.2,-3.8,107.6,33.4,0.2
5,00:00:10.4000000,10400,00:00:01.7000000,1700,00:06:05,365000,54786,8,Default,8333,...,Completed,535,583,10.4,1.7,365.0,5.8,106.5,33.8,0.6
6,00:00:04.6000000,4600,00:00:00.7000000,700,00:05:59.2000000,359200,54781,3,Default,8333,...,Completed,535,583,4.6,0.7,359.2,-5.9,108.2,33.3,0.1
7,00:00:10.5000000,10500,00:00:00.1000000,100,00:06:05.1000000,365100,54783,9,Default,8333,...,Completed,535,583,10.5,0.1,365.1,1.8,106.5,33.8,0.6
8,00:00:08.7000000,8700,00:00:02.1000000,2100,00:06:03.3000000,363300,54784,7,Default,8333,...,Completed,535,583,8.7,2.1,363.3,-2.4,107.0,33.6,0.4
9,00:00:11.1000000,11100,00:00:00.6000000,600,00:06:05.7000000,365700,54785,10,Default,8333,...,Completed,535,583,11.1,0.6,365.7,-99.0,106.3,33.9,0.7


In [73]:
wrc.getSplitTimes()

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages/8333/splittimes.json?rallyId=583


INFO:__main__:Updating split_times...


,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,stageTimeDuration,stageTimeDurationMs,startDateTime,startDateTimeLocal,stageId,eventId,rallyId
0,PT1M31S,91000,54778,2025-02-14T10:28:31,2025-02-14T11:28:31+01:00,13616,474825,00:05:58.5000000,358500.0,2025-02-14T10:27:00,2025-02-14T11:27:00+01:00,8333,535,583
1,PT3M4.4S,184400,54778,2025-02-14T10:30:04.4,2025-02-14T11:30:04.4+01:00,13621,474832,00:05:58.5000000,358500.0,2025-02-14T10:27:00,2025-02-14T11:27:00+01:00,8333,535,583
2,PT4M19.7S,259700,54778,2025-02-14T10:31:19.7,2025-02-14T11:31:19.7+01:00,13623,474840,00:05:58.5000000,358500.0,2025-02-14T10:27:00,2025-02-14T11:27:00+01:00,8333,535,583
3,PT1M29.1S,89100,54780,2025-02-14T10:31:29.1,2025-02-14T11:31:29.1+01:00,13616,474842,00:05:54.6000000,354600.0,2025-02-14T10:30:00,2025-02-14T11:30:00+01:00,8333,535,583
4,PT3M1.1S,181100,54780,2025-02-14T10:33:01.1,2025-02-14T11:33:01.1+01:00,13621,474847,00:05:54.6000000,354600.0,2025-02-14T10:30:00,2025-02-14T11:30:00+01:00,8333,535,583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,PT2M38.8S,158800,54838,2025-02-14T12:21:38.8,2025-02-14T13:21:38.8+01:00,13616,475471,00:08:54.6000000,534600.0,2025-02-14T12:19:00,2025-02-14T13:19:00+01:00,8333,535,583
173,PT4M55.4S,295400,54836,2025-02-14T12:21:55.4,2025-02-14T13:21:55.4+01:00,13623,475474,00:06:53.3000000,413300.0,2025-02-14T12:17:00,2025-02-14T13:17:00+01:00,8333,535,583
174,PT5M7.3S,307300,54837,2025-02-14T12:23:07.3,2025-02-14T13:23:07.3+01:00,13623,475482,00:07:10.2000000,430200.0,2025-02-14T12:18:00,2025-02-14T13:18:00+01:00,8333,535,583
175,PT4M50.6S,290600,54838,2025-02-14T12:23:50.6,2025-02-14T13:23:50.6+01:00,13621,475485,00:08:54.6000000,534600.0,2025-02-14T12:19:00,2025-02-14T13:19:00+01:00,8333,535,583


In [371]:
wrc.getStageOverallResults()

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages/8332/results.json?rallyId=583


INFO:__main__:Updating stage_overall...


,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId,eventId,rallyId
0,PT16.8S,16800,PT9.1S,9100,54777,PT0S,0,5,PT28M22.8S,1702800,PT28M22.8S,1702800,8332,535,583
1,PT0S,0,PT0S,0,54778,PT0S,0,1,PT28M6S,1686000,PT28M6S,1686000,8332,535,583
2,PT3.9S,3900,PT3.9S,3900,54779,PT0S,0,2,PT28M9.9S,1689900,PT28M9.9S,1689900,8332,535,583
3,PT5.8S,5800,PT1.9S,1900,54780,PT0S,0,3,PT28M11.8S,1691800,PT28M11.8S,1691800,8332,535,583
4,PT7.7S,7700,PT1.9S,1900,54781,PT0S,0,4,PT28M13.7S,1693700,PT28M13.7S,1693700,8332,535,583
5,PT16.9S,16900,PT0.1S,100,54782,PT0S,0,6,PT28M22.9S,1702900,PT28M22.9S,1702900,8332,535,583
6,PT58.3S,58300,PT2.6S,2600,54783,PT0S,0,10,PT29M4.3S,1744300,PT29M4.3S,1744300,8332,535,583
7,PT55.7S,55700,PT21.9S,21900,54784,PT0S,0,9,PT29M1.7S,1741700,PT29M1.7S,1741700,8332,535,583
8,PT33.8S,33800,PT0.9S,900,54785,PT0S,0,8,PT28M39.8S,1719800,PT28M39.8S,1719800,8332,535,583
9,PT32.9S,32900,PT16S,16000,54786,PT0S,0,7,PT28M38.9S,1718900,PT28M38.9S,1718900,8332,535,583


In [ ]:
## DONE ABOVE....

In [10]:
#
q = "SELECT name from season_rounds"
q = f'SELECT eventId, name FROM season_rounds WHERE name="Rally Sweden";'
wrc.query(q).iloc[0]["eventId"]

np.int64(535)

In [ ]:
getEventGroups(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/groups.json


,groupId,name
0,152,Rally1
1,98,Rally2
2,121,Rally3
3,100,Rally5


In [ ]:
getEventShakeDownTimes(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/shakedowntimes.json?shakedownNumber=1


,shakedownTimeId,eventId,entryId,runNumber,shakedownNumber,runDuration,runDurationMs
0,18735,535,54777,1,1,PT2M10.805S,130805
1,18754,535,54777,2,1,PT2M11.12S,131120
2,18791,535,54777,3,1,PT2M9.62S,129620
3,18824,535,54777,4,1,PT2M6.527S,126527
4,18731,535,54778,1,1,PT2M8.792S,128792
...,...,...,...,...,...,...,...
148,18878,535,54836,1,1,PT2M47.694S,167694
149,18863,535,54837,1,1,PT2M52.537S,172537
150,18855,535,54838,1,1,PT3M15.317S,195317
151,18867,535,54838,2,1,PT3M7.743S,187743


In [ ]:
rallyId, itineraryId, eventData_df, eventRallies_df, eventClasses_df = getEvent(eventId)
# rallyId = 584  # Safari WRC on eventId = 536 itineraryId = 1155
rallyId, itineraryId, eventData_df, eventRallies_df, eventClasses_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535.json


(583,
 1137,
            eventId  countryId  eventProfileId country          name  \
 countryId      535        215               2     215  Rally Sweden   
 name           535        215               2  Sweden  Rally Sweden   
 iso2           535        215               2      SE  Rally Sweden   
 iso3           535        215               2     SWE  Rally Sweden   
 
                       slug                   location   startDate  finishDate  \
 countryId  2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 name       2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 iso2       2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 iso3       2025-wrc-sweden  Umeå, Västerbotten County  2025-02-13  2025-02-16   
 
                  timeZoneId  ...    clerkOfTheCourse  \
 countryId  Europe/Stockholm  ...  Stig Rune Kjernsli   
 name       Europe/Stockholm  ...  Stig Rune Kjernsli   
 iso2       Europe/Stockholm  ...  Stig Ru

In [ ]:
getStageTimes(535, 583, 8332)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages/8332/stagetimes.json?rallyId=583


,stageTimeId,stageId,entryId,elapsedDurationMs,elapsedDuration,status,source,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,324396,8332,54780,643800,00:10:43.8000000,Completed,Default,1,0,00:00:00,0,00:00:00
1,324389,8332,54778,645600,00:10:45.6000000,Completed,Default,2,1800,00:00:01.8000000,1800,00:00:01.8000000
2,324431,8332,54781,646800,00:10:46.8000000,Completed,Default,3,3000,00:00:03,1200,00:00:01.2000000
3,324410,8332,54779,646800,00:10:46.8000000,Completed,Default,4,3000,00:00:03,0,00:00:00
4,324417,8332,54777,648500,00:10:48.5000000,Completed,Default,5,4700,00:00:04.7000000,1700,00:00:01.7000000
5,324403,8332,54782,650700,00:10:50.7000000,Completed,Default,6,6900,00:00:06.9000000,2200,00:00:02.2000000
6,324452,8332,54785,651700,00:10:51.7000000,Completed,Default,7,7900,00:00:07.9000000,1000,00:00:01
7,324445,8332,54784,654100,00:10:54.1000000,Completed,Default,8,10300,00:00:10.3000000,2400,00:00:02.4000000
8,324438,8332,54783,656600,00:10:56.6000000,Completed,Default,9,12800,00:00:12.8000000,2500,00:00:02.5000000
9,324424,8332,54786,658500,00:10:58.5000000,Completed,Default,10,14700,00:00:14.7000000,1900,00:00:01.9000000


In [ ]:
getEntries(eventId, rallyId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/rallies/583/entries.json


(   tags  entryId  eventId  driverId  codriverId  manufacturerId  entrantId  \
 0    []    54777      535       762         851              33        166   
 1    []    54778      535       534         553              84         91   
 2    []    54779      535       524         525              33        166   
 3    []    54780      535      4676        4945              33        166   
 4    []    54781      535       785        5507              84         91   
 ..  ...      ...      ...       ...         ...             ...        ...   
 57   []    54834      535     14234       17360              33       1148   
 58   []    54835      535     17361       17362              77       3801   
 59   []    54836      535      5525       17363              33       1380   
 60   []    54837      535       610         611              77       1379   
 61   []    54838      535       626        5545              69        572   
 
     groupId identifier     vehicleModel  entryLis

In [ ]:
from pandas import concat

itineraryLegs_df, itinerarySections_df, itineraryControls_df, itineraryStages_df = (
    getEventItineraries(eventId, itineraryId)
)
itineraryControls_df, itineraryStages_df

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/itineraries/1137.json


(    itinerarySectionId  itineraryLegId  order        name  controlId  eventId  \
 0                 3635            1506      1   Section 1      35201      535   
 1                 3635            1506      1   Section 1      35228      535   
 2                 3635            1506      1   Section 1      35229      535   
 3                 3635            1506      1   Section 1      35230      535   
 4                 3635            1506      1   Section 1      35202      535   
 ..                 ...             ...    ...         ...        ...      ...   
 76                3644            1509     10  Section 10      35226      535   
 77                3644            1509     10  Section 10      35279      535   
 78                3644            1509     10  Section 10      35280      535   
 79                3644            1509     10  Section 10      35281      535   
 80                3644            1509     10  Section 10      35227      535   
 
     stageId  

In [ ]:
getStages(eventId)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages.json


(    stageId  eventId  number                  name  distance     status  \
 0      8330      535       1         Umeå Sprint 1      5.16  Completed   
 1      8331      535       2          Bygdsiljum 1     28.27  Completed   
 2      8332      535       3       Andersvattnet 1     20.51  Completed   
 3      8333      535       4                Bäck 1     10.80  Completed   
 4      8334      535       5          Bygdsiljum 2     28.27  Completed   
 5      8335      535       6       Andersvattnet 2     20.51  Completed   
 6      8336      535       7                Bäck 2     10.80  Completed   
 7      8337      535       8         Umeå Sprint 2      5.16  Completed   
 8      8338      535       9              Vännäs 1     15.65  Completed   
 9      8339      535      10         Sarsjöliden 1     14.23  Completed   
 10     8340      535      11            Kolksele 1     16.06  Completed   
 11     8341      535      12              Vännäs 2     15.65  Completed   
 12     8342

In [ ]:
_getStageOverallResults(535, 583, 8332)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/535/stages/8332/results.json?rallyId=583


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev
0,54778,1686000,PT28M6S,0,PT0S,1686000,PT28M6S,1,0,PT0S,0,PT0S
1,54779,1689900,PT28M9.9S,0,PT0S,1689900,PT28M9.9S,2,3900,PT3.9S,3900,PT3.9S
2,54780,1691800,PT28M11.8S,0,PT0S,1691800,PT28M11.8S,3,5800,PT5.8S,1900,PT1.9S
3,54781,1693700,PT28M13.7S,0,PT0S,1693700,PT28M13.7S,4,7700,PT7.7S,1900,PT1.9S
4,54777,1702800,PT28M22.8S,0,PT0S,1702800,PT28M22.8S,5,16800,PT16.8S,9100,PT9.1S
5,54782,1702900,PT28M22.9S,0,PT0S,1702900,PT28M22.9S,6,16900,PT16.9S,100,PT0.1S
6,54786,1718900,PT28M38.9S,0,PT0S,1718900,PT28M38.9S,7,32900,PT32.9S,16000,PT16S
7,54785,1719800,PT28M39.8S,0,PT0S,1719800,PT28M39.8S,8,33800,PT33.8S,900,PT0.9S
8,54784,1741700,PT29M1.7S,0,PT0S,1741700,PT29M1.7S,9,55700,PT55.7S,21900,PT21.9S
9,54783,1744300,PT29M4.3S,0,PT0S,1744300,PT29M4.3S,10,58300,PT58.3S,2600,PT2.6S


In [ ]:
getControlTimes(534, 34754)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/534/controls/34754/controlTimes.json


,controlTimeId,controlId,entryId,dueDateTime,dueDateTimeLocal,actualDateTime,actualDateTimeLocal,absoluteActualDateTime,absoluteActualDateTimeLocal,source,status
0,1407418,34754,54668,2025-01-24T08:28:00,2025-01-24T09:28:00+01:00,2025-01-24T08:28:00,2025-01-24T09:28:00+01:00,2025-01-24T08:28:00,2025-01-24T09:28:00+01:00,Default,Completed
1,1407444,34754,54669,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,Default,Completed
2,1407470,34754,54670,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,Default,Completed
3,1407496,34754,54671,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,Default,Completed
4,1407522,34754,54672,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,Default,Completed
...,...,...,...,...,...,...,...,...,...,...,...
63,1408640,34754,54733,2025-01-24T10:04:00,2025-01-24T11:04:00+01:00,2025-01-24T10:04:00,2025-01-24T11:04:00+01:00,2025-01-24T10:04:00,2025-01-24T11:04:00+01:00,Default,Completed
64,1409030,34754,54734,2025-01-24T10:19:00,2025-01-24T11:19:00+01:00,2025-01-24T10:19:00,2025-01-24T11:19:00+01:00,2025-01-24T10:19:00,2025-01-24T11:19:00+01:00,Default,Completed
65,1409082,34754,54735,2025-01-24T10:21:00,2025-01-24T11:21:00+01:00,2025-01-24T10:21:00,2025-01-24T11:21:00+01:00,2025-01-24T10:21:00,2025-01-24T11:21:00+01:00,Default,Completed
66,1408900,34754,54736,2025-01-24T10:14:00,2025-01-24T11:14:00+01:00,2025-01-24T10:14:00,2025-01-24T11:14:00+01:00,2025-01-24T10:14:00,2025-01-24T11:14:00+01:00,Default,Completed


In [ ]:
getSplitTimes(534, 582, 8213)

https://p-p.redbull.com/rb-wrccom-lintegration-yv-prod/api/events/534/stages/8213/splittimes.json?rallyId=582


,startDateTime,startDateTimeLocal,stageTimeDurationMs,stageTimeDuration,splitPointTimeId,splitPointId,entryId,elapsedDurationMs,elapsedDuration,splitDateTime,splitDateTimeLocal
0,2025-01-24T08:31:00,2025-01-24T09:31:00+01:00,720600.0,00:12:00.6000000,467539,13346,54668,193500,PT3M13.5S,2025-01-24T08:34:13.5,2025-01-24T09:34:13.5+01:00
1,2025-01-24T08:34:00,2025-01-24T09:34:00+01:00,717800.0,00:11:57.8000000,467542,13346,54669,194700,PT3M14.7S,2025-01-24T08:37:14.7,2025-01-24T09:37:14.7+01:00
2,2025-01-24T08:37:00,2025-01-24T09:37:00+01:00,705000.0,00:11:45,467546,13346,54670,189800,PT3M9.8S,2025-01-24T08:40:09.8,2025-01-24T09:40:09.8+01:00
3,2025-01-24T08:40:00,2025-01-24T09:40:00+01:00,711000.0,00:11:51,467554,13346,54671,190700,PT3M10.7S,2025-01-24T08:43:10.7,2025-01-24T09:43:10.7+01:00
4,2025-01-24T08:43:00,2025-01-24T09:43:00+01:00,706400.0,00:11:46.4000000,467563,13346,54672,189200,PT3M9.2S,2025-01-24T08:46:09.2,2025-01-24T09:46:09.2+01:00
...,...,...,...,...,...,...,...,...,...,...,...
457,2025-01-24T10:22:00,2025-01-24T11:22:00+01:00,894000.0,00:14:54,467982,13353,54734,641300,PT10M41.3S,2025-01-24T10:32:41.3,2025-01-24T11:32:41.3+01:00
458,2025-01-24T10:23:00,2025-01-24T11:23:00+01:00,933700.0,00:15:33.7000000,467986,13353,54710,680000,PT11M20S,2025-01-24T10:34:20,2025-01-24T11:34:20+01:00
459,2025-01-24T10:24:00,2025-01-24T11:24:00+01:00,898800.0,00:14:58.8000000,467988,13353,54735,651200,PT10M51.2S,2025-01-24T10:34:51.2,2025-01-24T11:34:51.2+01:00
460,2025-01-24T10:25:00,2025-01-24T11:25:00+01:00,961200.0,00:16:01.2000000,467993,13353,54730,701500,PT11M41.5S,2025-01-24T10:36:41.5,2025-01-24T11:36:41.5+01:00


## Comparative testing

Compare old and new dataframes.


In [24]:
from wrc_rallydj.livetiming_api import WRCLiveTimingAPIClient

wrc_old = WRCLiveTimingAPIClient(use_cache=True, backend="memory", expire_after=60000)
wrc_old.initialise(eventName="WRC Rally Sweden")

Trying       id         guid                           title                location  \
0   K1Lg  WRC_2025_01              Rallye Monte-Carlo                  Monaco   
1   skC3  WRC_2025_02                WRC Rally Sweden                  sweden   
2   G2NQ  WRC_2025_03          WRC Safari Rally Kenya         Naivasha, Kenya   
3   zhuM  WRC_2025_04        WRC Rally Islas Canarias   Islas Canarias, Spain   
4   edGQ  WRC_2025_05  WRC Vodafone Rally de Portugal       Matosinhos, Porto   
5   k5hf  WRC_2025_06       WRC Rally Italia Sardegna         Olbia, Sardinia   
6   eaYx  WRC_2025_07  WRC EKO Acropolis Rally Greece                   Lamia   
7   aIpJ  WRC_2025_08         WRC Delfi Rally Estonia                 Estonia   
8   CaGI  WRC_2025_09         WRC Secto Rally Finland               Jyväskylä   
9   nMcx  WRC_2025_10          WRC Rally del Paraguay                Paraguay   
10  XpJ0  WRC_2025_11         WRC Rally Chile Bio Bío              Concepcion   
11  nQmX  WRC_2025_12

In [ ]:
xx = wrc_old.getPenalties()
xx.columns, xx.head()

(Index(['id', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
        'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
        'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
        'groupClass', 'rallyId', 'entryId', 'controlId', 'penaltyTime',
        'penaltyDuration', 'penaltyDurationMs', 'reason', 'control'],
       dtype='object'),
      id carNo                              driverId driverCountry  \
 0  7208   #66  917a7353-c28f-5fa4-b9f2-3afc0f6993e9       Belgium   
 1  7209   #70  d075fa9c-91dd-58a3-9a14-c24fe8fdb7f2          Peru   
 2  7210   #51  bb8301ac-ba00-57ff-ad20-0e03a535cb0c    Greece IOC   
 3  7211   #70  d075fa9c-91dd-58a3-9a14-c24fe8fdb7f2          Peru   
 4  7212   #41  5420dcab-71e0-5b63-a8f5-65ce359ed1e0        Turkey   
 
   driverCountryImage              driver  \
 0      Flags/BEL.png       Lyssia BAUDET   
 1      Flags/PER.png      Jorge MARTÍNEZ   
 2      Flags/GRE.png  Georgios VASILAKIS   
 3

In [64]:
import pandas as pd
import urllib
import re


class NOMIS_CONFIG:
    # TO DO implement cache to cache list of datasets and dimensions associated with datasets (except time/date?)

    def __init__(self):
        NOMIS_STUB = "https://www.nomisweb.co.uk/api/v01/dataset/"

        self.url = NOMIS_STUB
        self.codes = None
        self.metadata = {}

    def _url_encode(self, params=None):
        if params is not None and params != "" and params != {}:
            # params='?{}'.format( '&'.join( ['{}={}'.format(p,params[p]) for p in params] ) )
            params = "?{}".format(urllib.urlencode(params))
        else:
            params = ""
        return params

    def _describe_dataset(self, df):
        for row in df.iterrows():
            dfr = row[1]
            print(
                "{idx} - {name}: {description}\n".format(
                    idx=dfr["idx"], name=dfr["name"], description=dfr["description"]
                )
            )

    def _describe_metadata(self, idx, df, keys, pretty=True):
        if not pretty:
            for key in keys:
                print("---- {} ----".format(key))
                for row in df[key].iterrows():
                    dfr = row[1]
                    print(
                        "{dimension} - {description}: {value}".format(
                            dimension=dfr["dimension"],
                            description=dfr["description"],
                            value=dfr["value"],
                        )
                    )
        else:
            print(
                "The following dimensions are available for {idx} ({name}):\n".format(
                    idx=idx, name=self.dataset_lookup_property(idx, "name")
                )
            )
            for key in keys:
                items = [
                    "{} ({})".format(row[1]["description"], row[1]["value"])
                    for row in df[key].iterrows()
                ]
                print(" - {key}: {items}".format(key=key, items=", ".join(items)))

    def help_url(self, idx="NM_7_1"):
        metadata = self.nomis_code_metadata(idx)
        keys = metadata.keys()
        keys.remove("core")
        print(
            "Dataset {idx} ({name}) supports the following dimensions: {dims}.".format(
                idx=idx,
                dims=", ".join(keys),
                name=self.dataset_lookup_property(idx, "name"),
            )
        )

    def dataset_lookup_property(self, idx=None, prop=None):
        if idx is None or prop is None:
            return ""
        df = self.dataset_lookup(idx)

        if prop in df.columns:
            return str(df[prop][0])
        else:
            return ""

    def dataset_lookup(self, idx=None, dimensions=False, describe=False):
        ##dimensions used in sense of do we display them or not
        if self.codes is None:
            self.codes = self.nomis_codes_datasets(dimensions=True)

        if idx is not None:
            # Test if idx is a list or single string
            if isinstance(idx, str):
                idx = [idx]
            df = self.codes[self.codes["idx"].isin(idx)]
        else:
            df = self.codes[:]

        cols = df.columns.tolist()
        if not dimensions:
            for col in ["dimension", "concept"]:
                cols.remove(col)
        df = df[cols].drop_duplicates().reset_index(drop=True)
        if describe:
            self._describe_dataset(df)
        else:
            return df

    def _get_geo_from_postcode(self, postcode, areacode=None):
        # Set a default
        if areacode is None:
            areacode = "district"

        codemap = {"district": 486}

        if areacode in codemap:
            areacode = codemap[areacode]

        return "POSTCODE|{postcode};{code}".format(postcode=postcode, code=areacode)

    def _dimension_mapper(self, idx, dim, dims):
        """dims is a string of comma separated values for a particular dimension"""
        if dim is not None:
            sc = self._nomis_codes_dimension_grab(dim, idx, params=None)
            dimmap = dict(zip(sc["description"].astype(str), sc["value"]))
            keys = dimmap.keys()
            keys.sort(key=len, reverse=True)
            for s in keys:
                pattern = re.compile(s, re.IGNORECASE)
                dims = pattern.sub(str(dimmap[s]), str(dims))
        return dims

    def _sex_map(self, idx, sex):
        return self._dimension_mapper(idx, "sex", sex)

    def _get_geo_code_helper(self, helper):
        value = None
        desc = None

        # I am baking values in, but maybe they should be searched for and retrieved that way?
        if helper == "UK_WPC_2010":
            # UK Westminster Parliamentary Constituency
            value = "2092957697TYPE460"
        elif helper == "LA_district":
            value = "2092957697TYPE464"

        return value, desc

    def get_geo_code(
        self, value=None, desc=None, search=None, helper=None, chase=False
    ):
        # The semantics of this are quite tricky
        # value is a code for a geography, the thing searched within
        # desc identifies a description within a geography - on a match it takes you to this lower geography
        # search is term to search (free text search) with the descriptions of areas returned
        # helper is in place for shortcuts

        # Given a local authority code, eg 1946157281, a report can be previewed at:
        ##https://www.nomisweb.co.uk/reports/lmp/la/1946157281/report.aspx
        # default
        if helper is not None:
            value, desc = self._get_geo_code_helper(helper)
        if chase:
            chaser = self.nomis_codes_geog(geography=value)
            if search is not None:
                chasecands = chaser[chaser["description"].str.contains(search)][
                    ["description", "value"]
                ].values
            else:
                chasecands = chaser[["description", "value"]].values
            locs = []
            for chasecand in chasecands:
                locs.append(chasecand[1])
            if len(locs):
                value = ",".join(map(str, locs))

        geog = self.nomis_codes_geog(geography=value)
        if desc is not None:
            candidates = geog[["description", "value"]].values
            for candidate in candidates:
                if candidate[0] == desc:
                    geog = self.nomis_codes_geog(geography=candidate[1])

        if search is not None:
            retval = geog[geog["description"].str.contains(search)][
                ["description", "value"]
            ].values
        else:
            retval = geog[["description", "value"]].values

        return pd.DataFrame(retval, columns=["description", "geog"])

    def _get_datasets(self, search=None):
        url = "http://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json"
        if search is not None:
            url = "{url}{params}".format(
                url=url, params=self._url_encode({"search": search})
            )
        data = pd.read_json(url)
        return data

    def nomis_code_metadata(self, idx="NM_1_1", describe=None):
        if idx in self.metadata:
            metadata = self.metadata[idx]
        else:
            core = self.dataset_lookup(idx, dimensions=True)
            metadata = {"core": core}
            for dim in core["concept"].str.lower():
                metadata[dim] = self._nomis_codes_dimension_grab(dim, idx, params=None)
        self.metadata[idx] = metadata
        if describe == "all":
            keys = metadata.keys()
            keys.remove("core")
            self._describe_metadata(idx, metadata, keys)
        elif isinstance(describe, str) and describe in metadata.keys():
            self._describe_metadata(idx, metadata, [describe])
        elif isinstance(describe, list):
            self._describe_metadata(idx, metadata, describe)
        else:
            return metadata

    def nomis_codes_datasets(self, search=None, dimensions=False):
        # TO DO - by default, use local dataset list and search in specified cols;
        #  add additional parameter to force a search on API

        df = self._get_datasets(search)

        keyfamilies = df.loc["keyfamilies"]["structure"]
        if keyfamilies is None:
            return pd.DataFrame()

        datasets = []
        for keyfamily in keyfamilies["keyfamily"]:
            kf = {
                "agency": keyfamily["agencyid"],
                "idx": keyfamily["id"],
                "name": keyfamily["name"]["value"],
                "description": (
                    keyfamily["description"]["value"]
                    if "description" in keyfamily
                    else ""
                ),
                #'dimensions':[dimensions['codelist'] for dimensions in keyfamily['components']['dimension']]
            }

            if dimensions:
                for _dimensions in keyfamily["components"]["dimension"]:
                    kf["dimension"] = _dimensions["codelist"]
                    kf["concept"] = _dimensions["conceptref"]
                    datasets.append(kf.copy())
            else:
                datasets.append(kf.copy())

        return pd.DataFrame(datasets)

    def _nomis_codes_parser(self, url):
        jdata = pd.read_json(url)
        cl = jdata.loc["codelists"]["structure"]
        if cl is None:
            return pd.DataFrame()

        codes_data = []
        for codelist in cl["codelist"]:
            code_data = {
                "agencyid": codelist["agencyid"],
                "dataset": jdata.loc["header"]["structure"]["id"],
                "dimension": codelist["id"],
                "name": codelist["name"]["value"],
            }
            for code in codelist["code"]:
                code_data["description"] = code["description"]["value"]
                code_data["value"] = code["value"]
                codes_data.append(code_data.copy())
        return pd.DataFrame(codes_data)

    # Generic mininal constructor
    def _nomis_codes_url_constructor(self, dim, idx, params=None):
        # This doesn't cope with geography properly that can insert an element into the path?
        return "{nomis}{idx}/{dim}.def.sdmx.json{params}".format(
            nomis=self.url, idx=idx, dim=dim.lower(), params=self._url_encode(params)
        )

    def _nomis_codes_dimension_grab(self, dim, idx, params=None):
        url = self._nomis_codes_url_constructor(dim, idx, params=None)
        return self._nomis_codes_parser(url)

    # Set up shorthand functions to call particular dimensions
    # Select appropriate datsets as default to demo the call
    def nomis_codes_measures(self, idx="NM_1_1"):
        url = self._nomis_codes_url_constructor("measures", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_time(self, idx="NM_1_1"):
        url = self._nomis_codes_url_constructor("time", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_industry(self, idx="NM_21_1"):
        url = self._nomis_codes_url_constructor("industry", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_freq(self, idx="NM_1_1"):
        url = url = self._nomis_codes_url_constructor("freq", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_age_dur(self, idx="NM_7_1"):
        url = url = self._nomis_codes_url_constructor("age_dur", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_ethnicity(self, idx="NM_118_1"):
        url = url = self._nomis_codes_url_constructor("ethnicity", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_occupation(self, idx="NM_7_1"):
        url = url = self._nomis_codes_url_constructor("occupation", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_age(self, idx="NM_18_1"):
        url = url = self._nomis_codes_url_constructor("age", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_duration(self, idx="NM_18_1"):
        url = url = self._nomis_codes_url_constructor("duration", idx)
        return self._nomis_codes_parser(url)

    def nomis_codes_sex(self, idx="NM_1_1", geography=None):
        params = {}
        if geography is not None:
            params["geography"] = geography

        url = "{nomis}{idx}/sex.def.sdmx.json{params}".format(
            nomis=self.url, idx=idx, params=self._url_encode(params)
        )

        return self._nomis_codes_parser(url)

    def nomis_codes_geog(self, idx="NM_1_1", geography=None, search=None):
        params = {}
        if geography is not None:
            geog = "/{geog}".format(geog=geography)
        else:
            geog = ""

        if search is not None:
            params["search"] = search

        url = "{nomis}{idx}/geography{geog}.def.sdmx.json{params}".format(
            nomis=self.url, idx=idx, geog=geog, params=self._url_encode(params)
        )

        return self._nomis_codes_parser(url)

    def nomis_codes_items(self, idx="NM_1_1", geography=None, sex=None):
        sex = self._sex_map(idx, sex)
        params = {}

        if geography is not None:
            params["geography"] = geography
        if sex is not None:
            params["sex"] = sex

        url = "{nomis}{idx}/item.def.sdmx.json{params}".format(
            nomis=self.url, idx=idx, params=self._url_encode(params)
        )

        return self._nomis_codes_parser(url)

    # TO DO have a dataset_explain(idx) function that will print a description of a dataset,
    # summarise what dimensions are available, and the value they can take,
    # and provide a stub function usage example (with eligible parameters) to call it

    def _nomis_data_url(self, idx="NM_1_1", postcode=None, areacode=None, **kwargs):

        # TO DO
        # Add an explain=True parameter that will print a natural language summary of what the command is calling

        ###---Time/date info from nomis API docs---
        # Useful time options:
        ##"latest" - the latest available data for this dataset
        ##"previous" - the date prior to "latest"
        ##"prevyear" - the date one year prior to "latest"
        ##"first" - the oldest available data for this dataset
        ##Using the "time" concept you are limited to entering two dates,
        ##a start and end. All dates between these are returned.

        # date is more flexible for ranges
        ##With the "date" parameter you can specify relative dates,
        ##so for example if you wanted the latest date, three months and six months prior to that
        ##you could specify "date=latest,latestMINUS3,latestMINUS6".
        ##You can use ranges with the "date" parameter,
        ##e.g. if you wanted data for 12 months ago, together with all dates in the last six month
        ##up to latest you could specify "date=prevyear,latestMINUS5-latest".

        ##To illustrate the difference between using "date" and "time";
        ##if you specified "time=first,latest" in your URI you would get all dates from first to latest inclusive,
        ##whereas with "date=first,latest" your output would contain only the first and latest dates.

        metadata = self.nomis_code_metadata(idx)

        # HELPERS

        # Find geography from postcode
        if "geography" not in kwargs and postcode is not None:
            kwargs["geography"] = self._get_geo_from_postcode(postcode, areacode)

        # Map natural language dimension values to corresponding codes
        for dim in set(metadata.keys()).intersection(kwargs.keys()):
            kwargs[dim] = self._dimension_mapper(idx, dim, kwargs[dim])

        # Set a default time period to be latest
        if "date" not in kwargs and "time" not in kwargs:
            kwargs["time"] = "latest"

        # Set up a default projection for the returned columns
        cols = [
            "geography_code",
            "geography_name",
            "measures_name",
            "measures",
            "date_code",
            "date_name",
            "obs_value",
        ]

        for k in ["sex", "age", "item"]:
            if k in kwargs:
                cols.insert(len(cols) - 1, "{}_name".format(k))

        if "select" not in kwargs:
            kwargs["select"] = ",".join(cols)

        url = "{nomis}{idx}.data.csv{params}".format(
            nomis=self.url, idx=idx, params=self._url_encode(kwargs)
        )
        return url

    def _nomis_data(self, idx="NM_1_1", postcode=None, areacode=None, **kwargs):
        url = self._nomis_data_url(idx, postcode, areacode, **kwargs)

        df = pd.read_csv(url)
        df["_Code"] = idx
        return df